<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第2章：文本数据处理


本笔记本中使用的包：


In [1]:
#!pip install tiktoken

In [2]:
# ================================================================
# 模块功能说明：
# 本模块用于查看当前 Python 环境中安装的库（如 torch、tiktoken）的版本号。
# 这在调试环境或撰写教学笔记时非常重要，因为不同版本的库可能导致代码行为不同。
# ================================================================

# ------------------------------------------------
# 从 Python 内置模块 importlib.metadata 导入 version 函数
# ------------------------------------------------
# importlib.metadata 是 Python 3.8 及以上版本的标准库
# 用于访问当前环境中已安装的包的元数据，包括版本号、依赖信息、作者、许可证等。
#
# 函数 version(package_name)
# 作用：返回指定包的版本号，返回类型为字符串
# 参数：
#     - package_name：字符串类型，表示要查询的包名（必须是已安装的库）
#       示例："torch" 或 "tiktoken"
# 返回值：
#     - 字符串类型，例如 "2.3.0" 或 "0.6.0"
# 注意：
#     若查询不存在的包，会抛出 importlib.metadata.PackageNotFoundError 异常
# ------------------------------------------------
from importlib.metadata import version


# ------------------------------------------------
# 打印 PyTorch 库的版本号
# ------------------------------------------------
# 使用 print() 输出文本和变量结果
# 参数说明：
# - 第一个参数："torch version:" 是一个字符串，用于提示输出内容
# - 第二个参数：version("torch") 返回当前安装的 torch 版本号
# 输出示例："torch version: 2.3.1"
print("torch version:", version("torch"))


# ------------------------------------------------
# 打印 tiktoken 库的版本号
# ------------------------------------------------
# tiktoken 是 OpenAI 提供的高性能分词库，用于 GPT 模型的分词和 token 计数
# version("tiktoken") 获取当前安装的 tiktoken 库版本
# 输出示例："tiktoken version: 0.7.0"
print("tiktoken version:", version("tiktoken"))


torch version: 2.9.0+cpu
tiktoken version: 0.12.0


* 本章介绍如何进行数据准备与采样，以使输入数据为大型语言模型（LLM）做好“就绪”准备。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 理解词嵌入（Word Embeddings）


- 本节不包含任何代码


- 嵌入（embeddings）有多种形式；在本书中，我们关注文本嵌入（text embeddings）


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- 大型语言模型（LLMs）在高维空间中处理嵌入（即数千维度）  
- 由于我们无法可视化如此高维的空间（人类思维通常在 1、2 或 3 维），下图展示了一个二维嵌入空间


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本分词（Tokenizing text）


- 在本节中，我们对文本进行分词，这意味着将文本拆分为更小的单元，例如单个单词和标点符号


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 加载我们要处理的原始文本  
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) 是一篇公有领域的短篇小说


In [3]:
# ================================================================
# 模块功能说明：
# 本模块用于下载文本文件 "the-verdict.txt"。
# 如果文件已经存在，则不会重复下载。
# 使用 requests 库进行下载，比 urllib 更稳健，特别是存在 VPN 或网络限制时。
# ================================================================

# ------------------------------------------------
# 导入必要的 Python 库
# ------------------------------------------------
import os          # 用于操作文件路径和判断文件是否存在
import requests    # 用于发送 HTTP 请求，从网络获取文件内容


# ------------------------------------------------
# 判断文件是否已存在
# ------------------------------------------------
# os.path.exists(file_path) 用于检查指定路径的文件是否存在
# 如果文件不存在，则执行下载操作
if not os.path.exists("the-verdict.txt"):
    
    # ------------------------------------------------
    # 定义文件的下载 URL
    # ------------------------------------------------
    # Python 支持字符串拼接，这里分多行写更易读
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    
    # 定义本地保存路径
    file_path = "the-verdict.txt"

    # ------------------------------------------------
    # 发送 HTTP GET 请求下载文件
    # ------------------------------------------------
    # requests.get(url, timeout=30) 发送 GET 请求
    # 参数说明：
    # - url: 要下载文件的完整 URL
    # - timeout: 请求超时时间（秒），防止网络长时间阻塞
    response = requests.get(url, timeout=30)
    
    # ------------------------------------------------
    # 检查请求是否成功
    # ------------------------------------------------
    # response.raise_for_status() 会抛出异常，如果 HTTP 返回状态码不是 2xx
    # 这样可以保证只有在下载成功时才继续写入文件
    response.raise_for_status()
    
    # ------------------------------------------------
    # 将下载内容写入本地文件
    # ------------------------------------------------
    # 使用 with open(...) as f 打开文件，确保操作结束后自动关闭文件
    # 参数说明：
    # - file_path: 文件保存路径
    # - "wb": 以二进制写模式打开文件
    # - response.content: 获取 HTTP 响应内容（二进制）
    with open(file_path, "wb") as f:
        f.write(response.content)


# ------------------------------------------------
# 备注：
# ------------------------------------------------
# 原书中使用 urllib.request.urlretrieve 下载文件：
# import urllib.request
# urllib.request.urlretrieve(url, file_path)
# 但是 urllib 在某些环境（如使用 VPN）下可能出现网络协议问题
# requests 更现代、更稳健，并且使用方式更灵活


<br>

---

<br>

#### SSL 证书错误排查

- 一些读者在 VSCode 或 Jupyter 中运行 `urllib.request.urlretrieve` 时，报告出现 `ssl.SSLCertVerificationError: SSL: CERTIFICATE_VERIFY_FAILED` 错误。  
- 这通常意味着 Python 的证书包已过期。

**解决方法**

- 使用 Python ≥ 3.9；你可以通过执行以下代码检查 Python 版本：

```python
import sys
print(sys.__version__)
```
- 升级证书包：  
  - pip: `pip install --upgrade certifi`  
  - uv: `uv pip install --upgrade certifi`  
- 升级后重启 Jupyter 内核。  
- 如果在执行前面的代码单元时仍遇到 `ssl.SSLCertVerificationError`，请参阅 GitHub 上的讨论：[更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)

<br>

---

<br>


In [4]:
# ================================================================
# 模块功能说明：
# 本模块用于读取本地文本文件 "the-verdict.txt" 的内容
# 并统计字符总数，同时展示前 100 个字符作为示例
# ================================================================

# ------------------------------------------------
# 打开文件并读取内容
# ------------------------------------------------
# 使用 with open(...) as f 打开文件，确保操作结束后自动关闭文件
# 参数说明：
# - "the-verdict.txt": 要读取的文件路径
# - "r": 以只读模式打开文件
# - encoding="utf-8": 指定文件编码为 UTF-8，保证中文或特殊字符不会报错
#
# f.read(): 读取文件中所有内容，返回字符串
# 将内容保存到变量 raw_text 中
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()


# ------------------------------------------------
# 输出文件的字符总数
# ------------------------------------------------
# len(raw_text): 计算字符串长度，即文件中字符总数
print("Total number of character:", len(raw_text))


# ------------------------------------------------
# 输出文件前 100 个字符作为示例
# ------------------------------------------------
# raw_text[:99]: 切片操作，获取从索引 0 到 98 的字符（共 99 个字符）
# 用于快速查看文件开头内容，方便检查文件是否正确读取
print(raw_text[:99])


Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对这段文本进行分词并嵌入，以供 LLM 使用  
- 我们先基于一些简单示例文本开发一个简单的分词器，然后再将其应用到上面的文本  
- 以下正则表达式将根据空格进行分割


In [7]:
# ================================================================
# 模块功能说明：
# 本模块演示如何使用 Python 内置 re 模块进行字符串分割
# 并展示保留分隔符（这里是空格）的效果
# ================================================================

# ------------------------------------------------
# 导入正则表达式模块
# ------------------------------------------------
# re 模块提供正则表达式操作功能，包括搜索、匹配、替换和分割等
import re


# ------------------------------------------------
# 定义要处理的字符串
# ------------------------------------------------
text = "Hello, world. This, is a test."
# text 是一个普通字符串
# 示例内容中包含逗号、句号和空格，用于展示正则分割效果


# ------------------------------------------------
# 使用 re.split() 按空格分割字符串
# ------------------------------------------------
# re.split(pattern, string) 按正则 pattern 分割字符串
# 参数说明：
# - pattern: 正则表达式模式，这里是 r'(\s)'
#     - r'' 前缀表示原生字符串（raw string），避免反斜杠被转义
#     - (\s) 表示匹配任意空白字符（空格、制表符、换行等）
#       括号表示捕获组，分割结果会保留分隔符
# - string: 要处理的字符串，这里是 text
#
# 返回值：
# - 列表类型，包含原始子串和分隔符（空格）交替出现
result = re.split(r'(\s)', text)


# ------------------------------------------------
# 输出分割结果
# ------------------------------------------------
# 打印列表 result，方便查看分割效果
# 示例输出：
# ['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']
print(result)


['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 我们不仅希望按空格分割，还希望按逗号和句号分割，因此我们修改正则表达式以实现这一功能


In [8]:
# ================================================================
# 模块功能说明：
# 本模块演示如何使用 re.split() 按多种分隔符分割字符串
# 分隔符包括逗号、句号和空格，同时保留这些分隔符
# ================================================================

import re  # 导入正则表达式模块

# ------------------------------------------------
# 定义要处理的字符串
# ------------------------------------------------
text = "Hello, world. This, is a test."
# 示例字符串包含逗号、句号和空格，用于展示多分隔符分割效果

# ------------------------------------------------
# 使用 re.split() 按逗号、句号或空格分割字符串
# ------------------------------------------------
# re.split(pattern, string) 按正则 pattern 分割字符串
# 参数说明：
# - pattern: r'([,.]|\s)'
#     - r'' 表示原生字符串（raw string），避免反斜杠被转义
#     - [,.] 表示匹配字符集中的任意一个字符，这里是逗号 , 或句号 .
#     - \s 表示匹配任意空白字符（空格、制表符、换行等）
#     - | 表示“或”，即匹配 [,.] 或 \s
#     - () 括号表示捕获组，分割结果会保留匹配到的分隔符
# - string: 要处理的字符串，这里是 text
#
# 返回值：
# - 列表类型，包含原始子串和分隔符交替出现
result = re.split(r'([,.]|\s)', text)

# ------------------------------------------------
# 输出分割结果
# ------------------------------------------------
# 打印列表 result，方便查看分割效果
# 示例输出：
# ['Hello', ',', ' ', 'world', '.', ' ', 'This', ',', ' ', 'is', ' ', 'a', ' ', 'test', '.']
print(result)


['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 如我们所见，这会生成空字符串，因此我们需要将其移除


In [6]:
# ================================================================
# 模块功能说明：
# 本模块演示如何对列表进行清洗：
# 去掉每个元素的前后空白，并过滤掉空字符串
# ================================================================

# ------------------------------------------------
# 列表推导式处理列表 result
# ------------------------------------------------
# [item for item in result if item.strip()]
# 逐项解释：
# - item for item in result
#     遍历列表 result 中的每个元素，赋值给变量 item
# - item.strip()
#     - strip() 是字符串方法
#     - 去掉字符串前后空白字符（空格、制表符、换行等）
#     - 如果字符串去掉空白后长度为 0，则返回 False（在 if 中相当于 False）
# - if item.strip()
#     - 过滤条件：只有去掉空白后非空的字符串才会保留在新列表中
#
# 最终生成的新列表：
# - 每个元素都已去掉前后空白
# - 空字符串或只有空白字符的元素被移除
result = [item for item in result if item.strip()]

# ------------------------------------------------
# 输出清洗后的列表
# ------------------------------------------------
# 打印处理结果，方便检查
# 示例输出：
# ['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']
print(result)


['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 这看起来相当不错，但我们还需要处理其他类型的标点符号，例如句号、问号等


In [9]:
# ================================================================
# 模块功能说明：
# 本模块演示如何使用正则表达式将字符串按多种标点符号和空白字符分割，
# 并清洗结果，去掉空字符串和前后空白
# ================================================================

import re  # 导入正则表达式模块

# ------------------------------------------------
# 定义要处理的字符串
# ------------------------------------------------
text = "Hello, world. Is this-- a test?"
# 示例字符串包含逗号、句号、问号、破折号、空格等
# 用于展示复杂分隔符分割效果

# ------------------------------------------------
# 使用 re.split() 按多种标点符号和空白字符分割
# ------------------------------------------------
# re.split(pattern, string) 按正则表达式 pattern 分割字符串
# 参数说明：
# - pattern: r'([,.:;?_!"()\']|--|\s)'
#     - [] 表示字符集，匹配其中任意一个字符
#       这里匹配: , . : ; ? _ ! " ( ) '
#     - | 表示“或”，可以匹配多个条件
#     - -- 匹配两个连字符（破折号）
#     - \s 匹配任意空白字符（空格、制表符、换行等）
#     - () 括号表示捕获组，分割结果会保留匹配到的分隔符
# - string: 要处理的字符串，这里是 text
#
# 返回值：
# - 列表类型，包含原始子串和分隔符交替出现
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)

# ------------------------------------------------
# 清洗列表：去掉空字符串和前后空白
# ------------------------------------------------
# 列表推导式：
# - item.strip() 去掉每个元素的前后空白
# - if item.strip() 过滤掉空字符串（去掉空格后长度为0）
# 最终列表 result 中：
# - 每个元素都已去掉前后空白
# - 空字符串或只有空格的元素被移除
result = [item.strip() for item in result if item.strip()]

# ------------------------------------------------
# 输出清洗后的列表
# ------------------------------------------------
# 打印处理结果
# 示例输出：
# ['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']
print(result)


['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 这已经相当不错，现在我们可以将此分词方法应用到原始文本上


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [10]:
# ================================================================
# 模块功能说明：
# 本模块演示如何对文本 raw_text 进行预处理：
# 使用正则表达式将文本按标点符号和空白字符分割，
# 并清洗结果，去掉空字符串和前后空白
# ================================================================

import re  # 导入正则表达式模块

# ------------------------------------------------
# 使用 re.split() 按标点符号和空白字符分割文本
# ------------------------------------------------
# raw_text 是之前读取的整个文本文件内容
# re.split(pattern, string) 按正则表达式 pattern 分割字符串
# 参数说明：
# - pattern: r'([,.:;?_!"()\']|--|\s)'
#     - [] 表示字符集，匹配其中任意一个字符
#       这里匹配: , . : ; ? _ ! " ( ) '
#     - | 表示“或”，可以匹配多个条件
#     - -- 匹配破折号
#     - \s 匹配任意空白字符（空格、制表符、换行等）
#     - () 括号表示捕获组，分割结果会保留匹配到的分隔符
# - string: 要处理的文本，这里是 raw_text
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)

# ------------------------------------------------
# 清洗列表：去掉空字符串和前后空白
# ------------------------------------------------
# 列表推导式：
# - item.strip() 去掉每个元素的前后空白字符
# - if item.strip() 过滤掉空字符串（去掉空格或换行后长度为0）
# 最终列表 preprocessed 中：
# - 每个元素都是非空字符串
# - 保留原来的标点符号
preprocessed = [item.strip() for item in preprocessed if item.strip()]

# ------------------------------------------------
# 输出清洗后的前 30 个元素
# ------------------------------------------------
# 打印前 30 个元素作为示例，方便检查文本预处理效果
# 注意：这里使用切片 preprocessed[:30] 获取列表前 30 个元素
print(preprocessed[:30])


['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 让我们计算令牌（tokens）的总数


In [9]:
print(len(preprocessed))

4690


## 2.3 将令牌转换为令牌 ID


- 接下来，我们将文本令牌转换为令牌 ID，以便稍后通过嵌入层处理


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 从这些令牌中，我们现在可以构建一个词汇表，其中包含所有唯一的令牌


In [11]:
# ================================================================
# 模块功能说明：
# 本模块用于构建文本的词汇表（vocabulary）：
# - 去重
# - 排序
# - 统计词汇量
# ================================================================

# ------------------------------------------------
# 创建词汇表
# ------------------------------------------------
# set(preprocessed) 
# - 将列表 preprocessed 转换为集合（set）
# - 集合的特点是元素唯一，自动去重
# sorted(set(preprocessed))
# - 将去重后的集合按字母顺序排序
# - 返回一个列表，其中元素为去重且排序后的单词/符号
all_words = sorted(set(preprocessed))

# ------------------------------------------------
# 统计词汇表大小（词汇量）
# ------------------------------------------------
# len(all_words) 返回列表中元素个数，即词汇表大小
vocab_size = len(all_words)

# ------------------------------------------------
# 输出词汇表大小
# ------------------------------------------------
print(vocab_size)


1130


In [12]:
# ================================================================
# 模块功能说明：
# 本模块用于构建词汇表映射（token 到索引的字典）
# 方便后续将文本转换为整数序列，用于模型训练
# ================================================================

# ------------------------------------------------
# 使用字典推导式创建 token -> index 映射
# ------------------------------------------------
# {token: integer for integer, token in enumerate(all_words)}
# 逐项解释：
# - enumerate(all_words) 
#     - 对 all_words 列表进行枚举
#     - 返回每个元素的索引和元素本身，形式为 (索引, token)
#     - 示例: (0, 'Hello'), (1, ','), (2, 'World'), ...
# - for integer, token in enumerate(all_words)
#     - 遍历枚举对象，将索引赋值给 integer，元素赋值给 token
# - token: integer
#     - 在字典中，key 是 token（单词或符号）
#     - value 是 integer（索引）
# - 最终生成 vocab 字典，形式如：
#     {'Hello': 0, ',': 1, 'World': 2, ...}
vocab = {token: integer for integer, token in enumerate(all_words)}

# ------------------------------------------------
# 教学提示：
# ------------------------------------------------
# 1. vocab 可以将文本中的每个 token 转换为唯一整数索引
# 2. 构建 token->index 的字典是 NLP 模型文本编码的第一步
# 3. 对应的反向映射可以用：
#    inverse_vocab = {index: token for token, index in vocab.items()}


- 以下是该词汇表中的前 50 个条目：


In [13]:
# ================================================================
# 模块功能说明：
# 本模块演示如何遍历词汇表字典，并打印前 50 个 token->index 项
# ================================================================

# ------------------------------------------------
# 使用 enumerate 遍历 vocab 字典的 items()
# ------------------------------------------------
# vocab.items() 
# - 返回字典中所有 (key, value) 对的视图
# - 每个 item 是一个元组 (token, index)
#
# enumerate(vocab.items())
# - 对字典项进行枚举
# - 返回 (i, item)
#   - i: 当前迭代的索引（0 开始）
#   - item: 字典中的 (token, index) 元组
for i, item in enumerate(vocab.items()):
    
    # ------------------------------------------------
    # 打印当前字典项
    # ------------------------------------------------
    # item 是一个元组 (token, index)
    # 示例输出: ('Hello', 0)
    print(item)
    
    # ------------------------------------------------
    # 控制输出数量
    # ------------------------------------------------
    # if i >= 50: break
    # - 当打印 51 个元素后停止循环
    # - 避免输出太多内容，便于教学展示
    if i >= 50:
        break


('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下面通过一个小型词汇表演示短文本的分词过程：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将上述步骤整合到一个分词器类（tokenizer class）中


In [15]:
# ================================================================
# 模块功能说明：
# 本模块定义一个简单的分词器类 SimpleTokenizerV1
# 功能：
# 1. encode(text) - 将文本转换为整数序列
# 2. decode(ids) - 将整数序列还原为文本
# 适合 NLP 教学演示，理解 token -> index 映射与文本预处理
# ================================================================

import re  # 导入正则表达式模块，用于文本分割与空格处理

# ------------------------------------------------
# 定义分词器类
# ------------------------------------------------
class SimpleTokenizerV1:

    # ===========================================================
    # 初始化方法
    # ===========================================================
    # 参数：
    # - vocab: 字典类型，形式为 {token: index}，表示词汇表
    # 功能：
    # - 保存 token -> index 映射，便于将文本编码为整数
    # - 构建 index -> token 反向映射，便于解码
    def __init__(self, vocab):
        # 保存 token -> index 映射
        # vocab 是外部传入的字典，例如：{'Hello': 0, ',': 1, 'world': 2, ...}
        self.str_to_int = vocab

        # 构建 index -> token 的反向映射字典
        # 字典推导式 {i: s for s, i in vocab.items()} 遍历原字典
        # - s: token
        # - i: index
        # 最终结果为 {0: 'Hello', 1: ',', 2: 'world', ...}
        self.int_to_str = {i: s for s, i in vocab.items()}

    # ===========================================================
    # 文本编码方法
    # ===========================================================
    # 参数：
    # - text: 待编码的字符串，例如 "Hello, world!"
    # 返回值：
    # - ids: 对应整数索引列表，例如 [0, 1, 2, 3]
    def encode(self, text):
        # ------------------------------------------------
        # 1. 文本预处理
        # ------------------------------------------------
        # 使用正则表达式将文本按标点符号和空白字符分割
        # re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # 参数说明：
        # - [,.:;?_!"()\'] : 匹配逗号、句号、冒号、分号、问号、下划线、感叹号、双引号、括号、单引号
        # - --             : 匹配破折号
        # - \s             : 匹配任意空白字符（空格、制表符、换行符）
        # - ()             : 捕获组，保证分隔符保留在返回列表中
        # 返回值：
        # - preprocessed: 初步分割后的列表，包含 token 和分隔符
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        # ------------------------------------------------
        # 2. 去除空字符串和前后空白
        # ------------------------------------------------
        # 列表推导式 [item.strip() for item in preprocessed if item.strip()]
        # 逐步解释：
        # - item.strip() 去掉 token 前后的空白字符
        # - if item.strip() 过滤掉长度为 0 的元素（空字符串或仅空格）
        # 返回值：
        # - preprocessed: 清洗后的列表，仅包含有效 token 和标点
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        # ------------------------------------------------
        # 3. 将 token 转换为整数索引
        # ------------------------------------------------
        # 列表推导式 [self.str_to_int[s] for s in preprocessed]
        # 逐步解释：
        # - 遍历清洗后的 preprocessed 列表
        # - self.str_to_int[s]: 查找 token s 对应的整数索引
        # 返回值：
        # - ids: 整数序列列表，例如 [0, 1, 2, 3]
        ids = [self.str_to_int[s] for s in preprocessed]

        # 返回整数序列
        return ids

    # ===========================================================
    # 整数序列解码方法
    # ===========================================================
    # 参数：
    # - ids: 整数序列列表，例如 [0, 1, 2, 3]
    # 返回值：
    # - text: 还原后的字符串，例如 "Hello, world!"
    def decode(self, ids):
        # ------------------------------------------------
        # 1. 将整数序列转换回 token 列表
        # ------------------------------------------------
        # 列表推导式 [self.int_to_str[i] for i in ids]
        # 逐步解释：
        # - 遍历 ids 列表
        # - self.int_to_str[i]: 查找索引 i 对应的 token
        # - " ".join(...) 将 token 列表拼接为字符串，中间用空格分隔
        text = " ".join([self.int_to_str[i] for i in ids])

        # ------------------------------------------------
        # 2. 修复标点符号前的空格
        # ------------------------------------------------
        # re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        # 逐步解释：
        # - 正则 r'\s+([,.?!"()\'])'
        #   - \s+ 匹配一个或多个空白字符
        #   - ([,.?!"()\']) 捕获组，匹配标点符号
        # - 替换为 r'\1'
        #   - \1 表示捕获组中的标点符号
        #   - 相当于去掉标点符号前的空格
        # 返回值：
        # - text: 修正空格后的自然文本
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        # 返回解码后的文本
        return text


| 对比点      | `self.int_to_str = {i: s for s, i in vocab.items()}` | `enumerate(vocab.items())` |
| -------- | ---------------------------------------------------- | -------------------------- |
| **作用**   | 生成新的字典，将 key-value 反转                                | 遍历字典，给每一项加索引               |
| **返回值**  | 新字典 `{index: token}`                                 | 迭代器 `(i, (token, index))`  |
| **索引 i** | 来自原字典的 value                                         | 枚举序号，从 0 开始，与字典 value 无关   |
| **用途**   | 构建解码映射                                               | 打印或遍历字典，调试/教学用             |


- `encode` 函数将文本转换为 **token ID**  
- `decode` 函数则将 **token ID** 转换回文本


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用分词器将文本编码（即分词）为整数  
- 这些整数随后可以被嵌入（作为 LLM 的输入）


In [16]:
# ================================================================
# 模块功能说明：
# 本模块演示如何使用 SimpleTokenizerV1 对文本进行编码
# ================================================================

# ------------------------------------------------
# 1. 创建分词器实例
# ------------------------------------------------
# tokenizer = SimpleTokenizerV1(vocab)
# - vocab: 前面创建的 token->index 映射字典
# - SimpleTokenizerV1(vocab) 调用类的 __init__ 方法
#     - 保存 str_to_int 映射
#     - 构建 int_to_str 反向映射
# - tokenizer 是分词器对象，可以调用 encode 和 decode 方法
tokenizer = SimpleTokenizerV1(vocab)

# ------------------------------------------------
# 2. 定义待编码文本
# ------------------------------------------------
# text: 待编码的字符串
# 这里是多行字符串（Python 中 """ ... """ 支持多行）
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

# ------------------------------------------------
# 3. 编码文本
# ------------------------------------------------
# ids = tokenizer.encode(text)
# - 调用分词器对象的 encode 方法，将文本转换为整数序列
# 编码流程详细说明：
# 1) 文本预处理：
#    - 使用正则表达式将文本按标点符号和空白字符分割
#    - 捕获标点符号，保留在列表中
# 2) 清洗列表：
#    - 去掉空字符串和前后空白
# 3) token -> index 映射：
#    - 对每个 token 使用 self.str_to_int 查找对应整数索引
#    - 最终生成整数序列列表 ids
ids = tokenizer.encode(text)

# ------------------------------------------------
# 4. 打印整数序列
# ------------------------------------------------
# print(ids)
# - 输出编码后的整数序列
# - 每个整数对应原文本中的一个 token
# - 示例输出（根据 vocab 不同，实际值可能不同）：
#   [12, 7, 34, 56, 23, 89, 14, 5, 77, 3, ...]
print(ids)


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 我们可以将这些整数解码回文本


In [17]:
# ================================================================
# 模块功能说明：
# 本模块演示如何使用 SimpleTokenizerV1 的 decode 方法
# 将整数序列还原为可读文本
# ================================================================

# ------------------------------------------------
# 方法调用
# ------------------------------------------------
# tokenizer.decode(ids)
# - tokenizer: SimpleTokenizerV1 分词器实例
#   - 已经初始化时传入 vocab（token->index 映射）
# - decode(ids): 分词器方法，用于将整数序列还原为文本
# - ids: 待解码的整数序列列表
#   例如：
#     ids = [12, 7, 34, 56, 23, 89, 14, 5, 77, 3, ...]

# ------------------------------------------------
# decode 方法内部流程（详细讲解）
# ------------------------------------------------
# 1. 整数索引 → token 映射
#    [self.int_to_str[i] for i in ids]
#    - 遍历 ids 列表中的每个整数 i
#    - 使用 int_to_str 字典查找 i 对应的 token（单词或标点）
#    - 返回 token 列表，例如：
#        ['It', "'", 's', 'the', 'last', 'he', 'painted', ',', 'you', 'know', ',']
#
# 2. 拼接 token 为字符串
#    " ".join([...])
#    - 将 token 列表用空格 " " 连接成字符串
#    - 此时标点符号前可能有多余空格，例如：
#        "It ' s the last he painted , you know ,"
#
# 3. 去掉标点前多余空格
#    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
#    - 使用正则表达式替换标点前多余空格
#    - r'\s+([,.?!"()\'])' 匹配空格 + 标点
#    - r'\1' 表示只保留标点，删除前面空格
#    - 最终得到自然文本：
#        "It's the last he painted, you know,"

# ------------------------------------------------
# 执行解码
# ------------------------------------------------
decoded_text = tokenizer.decode(ids)

# ------------------------------------------------
# 输出解码后的文本
# ------------------------------------------------
# print(decoded_text)
# - decoded_text 是 ids 对应的原始文本（token化后的空格格式可能略有差异，但标点和单词保持一致）
print(decoded_text)


" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [18]:
# ================================================================
# 模块功能说明：
# 本模块演示如何使用 SimpleTokenizerV1 对文本进行“编码-解码”完整流程
# ================================================================

# ------------------------------------------------
# 示例文本
# ------------------------------------------------
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
# - text: 待处理的字符串，可以是多行文本
# - 示例中包含单引号、逗号、句号等标点符号
# - 可以用来演示分词器对标点和空格的处理

# ------------------------------------------------
# 编码-解码流程
# ------------------------------------------------
# tokenizer.encode(text)
# - 将原始文本 text 分割为 token，并映射为整数序列 ids
# - encode 方法内部流程：
#   1) 使用正则表达式将文本按标点符号和空白字符分割
#   2) 去掉空字符串和多余空白
#   3) 使用 str_to_int 映射将每个 token 转换为对应整数
# - 返回 ids 列表，例如：
#   ids = [12, 7, 34, 56, 23, 89, 14, 5, 77, 3, ...]

# tokenizer.decode(...)
# - 将整数序列 ids 还原为文本字符串
# - decode 方法内部流程：
#   1) 整数索引 → token 映射
#   2) token 列表 → 空格拼接字符串
#   3) 使用正则去掉标点前多余空格
# - 返回可读文本 decoded_text

# ------------------------------------------------
# 组合调用：先编码再解码
# ------------------------------------------------
decoded_text = tokenizer.decode(tokenizer.encode(text))
# - tokenizer.encode(text): 返回整数序列 ids
# - tokenizer.decode(...): 将 ids 还原为文本
# - decoded_text: 解码后的文本字符串
# - 这个组合展示了 encode 和 decode 的“正向-逆向”操作

# ------------------------------------------------
# 输出结果
# ------------------------------------------------
print("原始文本：")
print(text)
print("\n编码-解码后的文本：")
print(decoded_text)

# ================================================================
# 教学要点总结
# ================================================================
# 1. encode(text) 将文本 token 化并映射为整数序列
# 2. decode(ids) 将整数序列还原为文本
# 3. tokenizer.decode(tokenizer.encode(text)) 相当于对文本进行一次完整的“编码-解码”循环
# 4. 输出结果可以用于教学，观察标点和空格处理效果


原始文本：
"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride.

编码-解码后的文本：
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## 2.4 添加特殊上下文令牌


- 添加一些“特殊”令牌非常有用，例如用于表示未知单词或文本结尾


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些分词器使用特殊令牌来为 LLM 提供额外上下文  
- 常见的特殊令牌包括：
  - `[BOS]`（序列开始）标记文本的开始  
  - `[EOS]`（序列结束）标记文本的结束（通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书籍）  
  - `[PAD]`（填充）如果我们以大于 1 的批量大小训练 LLM（可能包含多条长度不同的文本；使用填充令牌可以将较短的文本填充到最长文本长度，使所有文本长度一致）  
  - `[UNK]` 表示不在词汇表中的单词

- 注意，GPT-2 并不需要上述任何特殊令牌，只使用 `<|endoftext|>` 令牌以降低复杂度  
- `<|endoftext|>` 类似于上文提到的 `[EOS]`  
- GPT 还使用 `<|endoftext|>` 进行填充（由于训练批量输入时通常使用掩码，填充令牌不会被关注，因此这些令牌的具体内容无关紧要）  
- GPT-2 不使用 `<UNK>` 处理词汇表外的单词，而是使用字节对编码（BPE）分词器，将单词拆分为子词单元，我们将在后续章节讨论


- 我们在两个独立文本源之间使用 `<|endoftext|>` 令牌：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 让我们看看对以下文本进行分词会发生什么：


In [19]:
# ================================================================
# 模块功能说明：
# 演示如何使用 SimpleTokenizerV1 对文本进行编码（encode）
# ================================================================

# ------------------------------------------------
# 1. 创建分词器实例
# ------------------------------------------------
tokenizer = SimpleTokenizerV1(vocab)
# - tokenizer: SimpleTokenizerV1 类的实例对象
# - vocab: 前面创建的 token -> index 字典，用于编码文本
# - 调用 __init__ 方法时：
#     1) 保存 str_to_int 映射（token -> index）
#     2) 构建 int_to_str 反向映射（index -> token）
# - tokenizer 对象可以调用 encode() 方法将文本转为整数序列
#   也可以调用 decode() 方法将整数序列还原为文本

# ------------------------------------------------
# 2. 定义待编码文本
# ------------------------------------------------
text = "Hello, do you like tea. Is this-- a test?"
# - text: 待编码的字符串文本
# - 示例文本包含：
#     - 单词: Hello, do, you, like, tea, Is, this, a, test
#     - 标点: 逗号, 句号, 问号, 破折号 (--)

# ------------------------------------------------
# 3. 编码文本
# ------------------------------------------------
tokenizer.encode(text)
# - 调用 tokenizer.encode() 方法
# - 内部流程：
#     1) 文本预处理：
#         - 使用正则表达式 re.split(r'([,.:;?_!"()\']|--|\s)', text)
#         - 将文本按标点符号、破折号、空格分割，并保留标点
#     2) 清理列表：
#         - 列表推导式 [item.strip() for item in preprocessed if item.strip()]
#         - 去掉空白字符或空字符串
#     3) token -> index 映射：
#         - [self.str_to_int[s] for s in preprocessed]
#         - 将每个 token 转换为整数索引
# - 返回整数列表 ids，例如：
#     [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, ...]

# ------------------------------------------------
# 教学要点总结
# ------------------------------------------------
# 1. encode() 是 NLP 模型输入的关键步骤，将文本转为整数序列
# 2. 正则表达式可以处理标点符号、空格和破折号
# 3. 返回的整数序列可以直接用于神经网络模型输入
# 4. 本示例展示了如何将文本 token 化并编码为整数


KeyError: 'Hello'

- 上述操作会报错，因为单词 "Hello" 不在词汇表中  
- 为了处理这种情况，我们可以向词汇表中添加特殊令牌，例如 `"<|unk|>"`，用于表示未知单词  
- 既然我们已经在扩展词汇表，可以再添加一个名为 `"<|endoftext|>"` 的令牌，它在 GPT-2 训练中用于表示文本结束（同时也用于连接文本，例如当训练数据集包含多篇文章、书籍等时）


In [20]:
# ================================================================
# 模块功能说明：
# 构建文本的词汇表（vocabulary）并为每个 token 分配唯一整数索引
# ================================================================

# ------------------------------------------------
# 1. 获取所有唯一 token 并排序
# ------------------------------------------------
all_tokens = sorted(list(set(preprocessed)))
# - preprocessed: 已经经过预处理的 token 列表（文本被分割、去空白）
#   例如：['Hello', ',', 'do', 'you', 'like', 'tea', '.', 'Is', 'this', '--', 'a', 'test']
# - set(preprocessed):
#   - 将列表转换为集合，去掉重复的 token
#   - 集合中元素无序
# - list(set(preprocessed)):
#   - 将集合转换回列表，便于排序和索引
# - sorted(...):
#   - 对 token 列表进行字母顺序排序
#   - 确保每次生成的 vocab 顺序一致
# - all_tokens 示例输出：
#   ['--', '.', ',', 'Is', 'Hello', 'a', 'do', 'like', 'tea', 'test', 'this', 'you']

# ------------------------------------------------
# 2. 添加特殊 token
# ------------------------------------------------
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
# - extend 方法将列表元素逐个添加到 all_tokens 末尾
# - 特殊 token 用途：
#     1) "<|endoftext|>": 文本结束标记
#     2) "<|unk|>": 未知 token（在词汇表中不存在的单词）
# - 添加特殊 token 后，all_tokens 列表完整，包含所有实际 token 和特殊 token

# ------------------------------------------------
# 3. 构建 token -> index 映射字典
# ------------------------------------------------
vocab = {token: integer for integer, token in enumerate(all_tokens)}
# - 字典推导式用于生成词汇表映射
# - enumerate(all_tokens)：
#     - 遍历 all_tokens 列表
#     - 返回索引（integer）和对应 token
#     - 示例：
#         0 -> '--'
#         1 -> '.'
#         2 -> ','
#         ...
# - {token: integer for integer, token in enumerate(all_tokens)}：
#     - 将 token 作为字典 key，整数索引作为 value
#     - 构建 token -> index 映射
# - vocab 示例：
#     {
#         '--': 0,
#         '.': 1,
#         ',': 2,
#         'Is': 3,
#         'Hello': 4,
#         'a': 5,
#         'do': 6,
#         'like': 7,
#         'tea': 8,
#         'test': 9,
#         'this': 10,
#         'you': 11,
#         '<|endoftext|>': 12,
#         '<|unk|>': 13
#     }

# ------------------------------------------------
# 教学要点总结
# ------------------------------------------------
# 1. 使用 set 去重，确保每个 token 唯一
# 2. sorted 保证词汇表顺序固定，便于教学和调试
# 3. 添加特殊 token 是 NLP 模型的标准做法
# 4. enumerate + 字典推导式生成 token -> index 映射，方便后续编码


In [21]:
# ================================================================
# 模块功能说明：
# 计算词汇表（vocab）中包含的 token 数量
# ================================================================

# ------------------------------------------------
# 1. vocab.items()
# ------------------------------------------------
# - vocab: 之前创建的 token -> index 映射字典
# - vocab.items() 返回字典的所有键值对（key-value tuple）
#   - 格式为可迭代对象，每个元素是 (token, index) 元组
#   - 示例：
#       dict_items([('--', 0), ('.', 1), (',', 2), ('Is', 3), ...])
# - 这个方法可以遍历词汇表的每个 token 和对应索引

# ------------------------------------------------
# 2. len(...)
# ------------------------------------------------
# - len() 内置函数，用于计算可迭代对象元素的数量
# - len(vocab.items()) 返回 vocab 字典中键值对的数量
# - 也就是词汇表中不同 token 的总数，包括特殊 token
# - 示例输出：
#     如果 vocab 中有 14 个 token，则 len(vocab.items()) = 14

# ------------------------------------------------
# 3. 教学用法
# ------------------------------------------------
# 这行代码可以帮助学生：
# 1) 直观了解词汇表大小
# 2) 检查是否包含特殊 token，如 "<|endoftext|>", "<|unk|>"
# 3) 为编码和模型输入维度提供参考
vocab_size = len(vocab.items())

# ------------------------------------------------
# 4. 输出词汇表大小
# ------------------------------------------------
print("词汇表总大小：", vocab_size)
# - 输出示例：
#   词汇表总大小： 14


词汇表总大小： 1132


In [22]:
# ================================================================
# 模块功能说明：
# 演示如何查看词汇表（vocab）中最后几个 token 及其索引
# ================================================================

# ------------------------------------------------
# 1. 遍历词汇表最后 5 个元素
# ------------------------------------------------
for i, item in enumerate(list(vocab.items())[-5:]):
    # vocab.items(): 返回字典的所有键值对，格式为 dict_items([(token, index), ...])
    # list(vocab.items()): 将 dict_items 转换为列表，以便支持切片操作
    # [-5:]: 切片操作，获取列表最后 5 个元素
    # enumerate(...):
    #   - 遍历切片后的列表
    #   - i 是循环索引（从 0 开始）
    #   - item 是当前元素，即 (token, index) 元组
    #     例如：('<|endoftext|>', 12)
    
    # ------------------------------------------------
    # 2. 输出当前 token 和索引
    # ------------------------------------------------
    print(item)
    # - 输出格式为 (token, index)
    # - 示例输出：
    #     ('this', 10)
    #     ('you', 11)
    #     ('<|endoftext|>', 12)
    #     ('<|unk|>', 13)
    # - 可用于教学观察词汇表末尾的特殊 token 或最后几个普通 token

# ------------------------------------------------
# 教学要点总结
# ------------------------------------------------
# 1. list(vocab.items())[-5:] 可以快速获取词汇表最后几个 token
# 2. enumerate 可以同时获得索引和元素，便于在循环中操作或打印
# 3. 输出内容帮助学生直观理解词汇表结构和特殊 token 的位置


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 我们还需要相应地调整分词器，以便它知道何时以及如何使用新的 `<unk>` 令牌


In [23]:
# ================================================================
# 模块功能说明：
# SimpleTokenizerV2 类实现了一个改进的简单分词器
# 相比 V1，增加了对未知 token 的处理
# ================================================================

class SimpleTokenizerV2:
    # ------------------------------------------------
    # 1. 初始化方法
    # ------------------------------------------------
    def __init__(self, vocab):
        # vocab: 传入的词汇表字典，token -> index
        # self.str_to_int: 保存 token -> index 映射
        self.str_to_int = vocab
        
        # self.int_to_str: 生成 index -> token 反向映射
        # 字典推导式 {i:s for s,i in vocab.items()}：
        # - 遍历 vocab.items() 返回 (token, index)
        # - 交换顺序，将 index 作为 key，token 作为 value
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    # ------------------------------------------------
    # 2. 编码方法
    # ------------------------------------------------
    def encode(self, text):
        # preprocessed: 使用正则分割文本，捕获标点和空白
        # re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # - 括号内的模式表示捕获组，匹配标点符号、破折号和空格
        # - 捕获组会保留在分割结果列表中
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        
        # 去掉空白或空字符串，并去除首尾空格
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        
        # 处理未知 token：
        # - 遍历 preprocessed
        # - 如果 token 存在于 str_to_int 中，则保持原 token
        # - 否则替换为 "<|unk|>"，表示未知 token
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        # token -> index 映射
        # - 遍历 preprocessed 列表
        # - 使用 str_to_int 将每个 token 转为整数索引
        ids = [self.str_to_int[s] for s in preprocessed]
        
        # 返回编码后的整数序列 ids
        return ids
        
    # ------------------------------------------------
    # 3. 解码方法
    # ------------------------------------------------
    def decode(self, ids):
        # ids: 待解码的整数序列列表
        # [self.int_to_str[i] for i in ids]:
        # - 将每个整数索引映射回 token
        # - 返回 token 列表
        text = " ".join([self.int_to_str[i] for i in ids])
        # - 用空格拼接 token 列表
        # - 此时标点前可能存在多余空格
        
        # 使用正则去掉标点前的多余空格
        # re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        # - 匹配空格后跟标点符号的情况
        # - \1 表示保留标点，删除空格
        # - 返回自然文本
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        
        # 返回解码后的文本字符串
        return text

# ================================================================
# 教学要点总结
# ================================================================
# 1. SimpleTokenizerV2 是 V1 的升级版，增加了对未知 token 的处理
# 2. encode 方法：
#    - 分割文本 → 去空白 → 替换未知 token → 转为整数序列
# 3. decode 方法：
#    - 整数序列 → token 列表 → 拼接字符串 → 修正标点空格
# 4. "<|unk|>" 可以保证模型在遇到词汇表外的 token 时不会报错


- 让我们尝试使用修改后的分词器对文本进行分词：


In [24]:
# ================================================================
# 模块功能说明：
# 演示如何使用 SimpleTokenizerV2 构建分词器实例并处理多段文本
# ================================================================

# ------------------------------------------------
# 1. 创建分词器实例
# ------------------------------------------------
tokenizer = SimpleTokenizerV2(vocab)
# - tokenizer: SimpleTokenizerV2 类的实例
# - vocab: 之前创建的 token -> index 映射字典
# - 实例化时会初始化：
#     - self.str_to_int: token -> index 映射
#     - self.int_to_str: index -> token 反向映射

# ------------------------------------------------
# 2. 定义多段文本
# ------------------------------------------------
text1 = "Hello, do you like tea?"
# - text1: 第一段文本字符串

text2 = "In the sunlit terraces of the palace."
# - text2: 第二段文本字符串

# ------------------------------------------------
# 3. 合并多段文本
# ------------------------------------------------
text = " <|endoftext|> ".join((text1, text2))
# - tuple (text1, text2): 将两个文本组合成元组，作为 join 的可迭代对象
# - " <|endoftext|> ".join(...):
#     - 在每段文本之间插入特殊 token "<|endoftext|>" 作为段落/文本分隔符
#     - 生成新的字符串 text：
#       "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."
# - 这样做可以让模型或分词器在处理多段文本时知道段落边界

# ------------------------------------------------
# 4. 输出合并后的文本
# ------------------------------------------------
print(text)
# - 输出示例：
#   Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
# - 教学用途：
#   1) 展示如何用特殊 token 分隔多段文本
#   2) 便于后续进行 encode 编码，保留段落信息


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [25]:
# ================================================================
# 模块功能说明：
# 演示如何使用 SimpleTokenizerV2 对合并后的多段文本进行编码（encode）
# ================================================================

# ------------------------------------------------
# 1. 方法调用
# ------------------------------------------------
tokenizer.encode(text)
# - tokenizer: SimpleTokenizerV2 类实例
#   - 已经用 vocab 初始化，包含 str_to_int 映射（token -> index）
# - encode(text): 分词器方法，用于将文本 text 转换为整数序列 ids
# - text: 合并后的多段文本，段落间用特殊 token "<|endoftext|>" 分隔
#   例如：
#     "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."

# ------------------------------------------------
# 2. encode 方法内部流程（详细解析）
# ------------------------------------------------
# 1) 文本分割：
#    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
#    - 按标点符号、破折号和空格分割文本
#    - 保留捕获的标点符号
#
# 2) 去掉空白：
#    preprocessed = [item.strip() for item in preprocessed if item.strip()]
#    - 去掉首尾空格
#    - 去掉空字符串或仅空格的 token
#
# 3) 处理未知 token：
#    preprocessed = [
#        item if item in self.str_to_int else "<|unk|>" for item in preprocessed
#    ]
#    - 如果 token 在词汇表中，保留原 token
#    - 如果 token 不在词汇表中，替换为 "<|unk|>"
#    - 特别注意："<|endoftext|>" 已经在 vocab 中，确保不会被替换为 "<|unk|>"
#
# 4) token -> index 映射：
#    ids = [self.str_to_int[s] for s in preprocessed]
#    - 将每个 token 映射为对应整数索引
#    - 返回整数序列 ids，便于作为模型输入

# ------------------------------------------------
# 3. 输出编码结果
# ------------------------------------------------
ids = tokenizer.encode(text)
print("编码后的整数序列 ids：")
print(ids)
# - ids 是整数列表
# - 可以直接用于模型输入
# - 特殊 token "<|endoftext|>" 对应特定整数索引
# - 教学用途：观察文本被分割、token 映射为整数的全过程

# ================================================================
# 教学要点总结
# ================================================================
# 1. encode 方法将文本转换为整数序列，是 NLP 模型输入的核心步骤
# 2. 支持多段文本和特殊 token "<|endoftext|>"
# 3. 内部流程：
#    分割文本 → 去空白 → 处理未知 token → token 映射为整数
# 4. 输出 ids 可以用来理解 token 化和编码的效果


编码后的整数序列 ids：
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [28]:
# ================================================================
# 模块功能说明：
# 演示对多段文本进行编码后立即解码的完整流程
# ================================================================

# ------------------------------------------------
# 1. 组合调用 encode 和 decode
# ------------------------------------------------
#tokenizer.decode(tokenizer.encode(text))
# - tokenizer.encode(text):
#     1) 将文本 text 分割成 token
#     2) 去掉空白和空字符串
#     3) 对未知 token 替换为 "<|unk|>"
#     4) 将 token 映射为整数索引，返回 ids 列表
# - tokenizer.decode(...):
#     1) 将整数索引 ids 转回 token
#     2) 用空格拼接 token 列表
#     3) 使用正则去掉标点前多余空格
#     4) 返回可读文本字符串

# ------------------------------------------------
# 2. 内部流程示例（假设 text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."）
# ------------------------------------------------
# 1) encode(text) 生成整数序列 ids：
#    例如：
#    ids = [4, 2, 6, 11, 7, 8, 3, 12, 10, 5, 15, ...]
#    - 其中 12 对应 "<|endoftext|>"，其他整数对应词汇表中的 token
# 2) decode(ids) 将整数序列还原：
#    - ids -> token 列表
#    - token 列表 -> 空格拼接字符串
#    - 去掉标点前多余空格
#    - 最终得到与原文本非常接近的文本，保留 "<|endoftext|>" 作为段落分隔符

# ------------------------------------------------
# 3. 输出结果
# ------------------------------------------------
decoded_text = tokenizer.decode(tokenizer.encode(text))
print("原始文本：")
print(text)
print("\n编码-解码后的文本：")
print(decoded_text)

# ------------------------------------------------
# 4. 教学要点总结
# ------------------------------------------------
# 1. encode -> decode 组合展示了分词器的完整流程
# 2. 多段文本和特殊 token "<|endoftext|>" 可以被正确处理
# 3. 输出结果可用于教学观察：
#    - token 化和整数映射
#    - 解码后的文本是否接近原文
# 4. 对于词汇表外的 token，会被替换为 "<|unk|>" 并解码回 "<|unk|>"


原始文本：
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.

编码-解码后的文本：
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## 2.5 字节对编码（BytePair Encoding, BPE）


- GPT-2 使用字节对编码（BytePair Encoding, BPE）作为其分词器  
- 它允许模型将不在预定义词汇表中的单词拆分为更小的子词单元甚至单个字符，从而处理词汇表外的单词  
- 例如，如果 GPT-2 的词汇表中没有单词 "unfamiliarword"，它可能会将其分词为 ["unfam", "iliar", "word"] 或其他子词拆分方式，这取决于训练时的 BPE 合并规则  
- 原始 BPE 分词器可在此找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)  
- 在本章中，我们使用 OpenAI 开源库 [tiktoken](https://github.com/openai/tiktoken) 提供的 BPE 分词器，该库的核心算法用 Rust 实现，以提升计算性能  
- 我在 [./bytepair_encoder](../02_bonus_bytepair-encoder) 中创建了一个笔记本，对比了这两种实现（在示例文本上 tiktoken 大约快 5 倍）


In [25]:
# pip install tiktoken

In [29]:
# ================================================================
# 模块功能说明：
# 演示如何导入 tiktoken 库并查看其版本号
# ================================================================

# ------------------------------------------------
# 1. 导入模块
# ------------------------------------------------
import importlib
# - importlib: Python 内置库，用于动态导入模块、获取模块元数据等
# - 本例中用于获取已安装包的版本号

import tiktoken
# - tiktoken: OpenAI 提供的高性能分词库
# - 可用于 GPT 模型的分词（tokenization）和 token 计数

# ------------------------------------------------
# 2. 打印 tiktoken 版本
# ------------------------------------------------
print("tiktoken version:", importlib.metadata.version("tiktoken"))
# - importlib.metadata.version("tiktoken"):
#     - 获取当前安装的 tiktoken 库的版本号（字符串类型）
#     - 例如："0.7.0"
# - print(...):
#     - 输出示例：
#       tiktoken version: 0.7.0
# - 教学用途：
#     1) 确认所用 tiktoken 版本，便于复现代码
#     2) 避免因版本差异导致 API 或分词结果不同


tiktoken version: 0.12.0


In [30]:
# ================================================================
# 模块功能说明：
# 使用 tiktoken 创建 GPT-2 编码器（tokenizer）
# ================================================================

# ------------------------------------------------
# 1. 创建 GPT-2 编码器
# ------------------------------------------------
tokenizer = tiktoken.get_encoding("gpt2")
# - tiktoken.get_encoding("gpt2"):
#     - 调用 tiktoken 库的 get_encoding 函数
#     - 参数 "gpt2": 指定编码器类型为 GPT-2 使用的 BPE (Byte-Pair Encoding)
# - 返回值：
#     - tokenizer 对象，用于文本的编码（encode）和解码（decode）
#     - 支持方法：
#         1) encode(text) -> 将文本转换为 token id 列表
#         2) decode(ids) -> 将 token id 列表还原为文本
#         3) encode_ordinary(text) / decode_ordinary(text) 等高级方法
# - 教学用途：
#     1) 演示如何获取官方 GPT-2 分词器
#     2) 准备后续文本编码和 token 数量统计
#     3) 与 SimpleTokenizerV2 对比，展示官方 BPE 分词器和自定义分词器的区别


In [31]:
# ================================================================
# 模块功能说明：
# 使用 tiktoken 的 GPT-2 编码器对文本进行编码（encode）
# ================================================================

# ------------------------------------------------
# 1. 定义待编码文本
# ------------------------------------------------
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
# - text: 待编码的字符串文本
# - 注意：
#     1) 使用了 Python 的括号和字符串拼接特性
#        - 两个字符串直接放在括号内，会自动连接
#        - "terraces" 和 "of someunknownPlace." 会被合并为一个连续字符串
#     2) 文本中包含：
#        - 普通单词: Hello, do, you, like, tea, In, the, sunlit, terraces, of, someunknownPlace
#        - 标点: , ? .
#        - 特殊 token: <|endoftext|>，用于指示文本结束或段落分隔

# ------------------------------------------------
# 2. 编码文本
# ------------------------------------------------
integers = tokenizer.encode(
    text,
    allowed_special={"<|endoftext|>"}
)
# - tokenizer: tiktoken.get_encoding("gpt2") 返回的编码器对象
# - encode(text, allowed_special={...}) 方法：
#     1) 将文本 text 转换为 token id 列表（整数序列）
#     2) allowed_special: 指定允许保留的特殊 token
#        - "<|endoftext|>": 会被作为单个 token 保留，不会拆分或报错
#     3) 遇到其他非词汇表 token 会根据 GPT-2 BPE 规则拆分为子 token
# - 返回值 integers：
#     - 一个整数列表，每个元素表示对应 token 的 id
#     - 示例输出可能类似：
#       [15496, 11, 703, 345, 284, 30, 50256, 315, 262, 12850, 2394, 13]

# ------------------------------------------------
# 3. 输出编码结果
# ------------------------------------------------
print(integers)
# - 输出整数列表
# - 教学用途：
#     1) 观察文本被拆分为 GPT-2 token 的过程
#     2) 了解特殊 token "<|endoftext|>" 被保留为单个 token
#     3) 与自定义 SimpleTokenizer 对比，展示官方 BPE 分词器处理未知单词的方式
#        - 例如 "someunknownPlace" 会被拆分为多个子 token


[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [32]:
# ================================================================
# 模块功能说明：
# 使用 tiktoken 的 GPT-2 编码器对整数 token id 列表进行解码（decode）
# ================================================================

# ------------------------------------------------
# 1. 解码整数序列
# ------------------------------------------------
strings = tokenizer.decode(integers)
# - tokenizer: tiktoken.get_encoding("gpt2") 返回的编码器对象
# - decode(integers) 方法：
#     1) 将整数列表 integers 转换回文本字符串
#     2) 每个整数对应一个 token id
#     3) 解码器会按照 GPT-2 BPE 规则将 token id 转回对应的子词或单词
#     4) 特殊 token "<|endoftext|>" 会被恢复为文本 "<|endoftext|>"
# - 返回值 strings：
#     - 解码后的字符串文本
#     - 示例：
#       "Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace."

# ------------------------------------------------
# 2. 输出解码结果
# ------------------------------------------------
print(strings)
# - 输出文本字符串
# - 教学用途：
#     1) 对比原始文本和解码文本
#     2) 观察 GPT-2 BPE 分词器处理未知单词和特殊 token 的效果
#     3) 理解 encode -> decode 的可逆性（除了 BPE 可能对空格有微调）
# - 注意：
#     - 在 BPE 解码中，可能出现原始空格与 token 拼接的差异
#     - 例如 "terracesof" 是原文本 "terraces of" 被拆分为子 token 后重新拼接的结果


Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE 分词器将未知单词拆分为子词和单个字符：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 使用滑动窗口进行数据采样


- 我们训练 LLM 以一次生成一个单词，因此需要相应地准备训练数据，其中序列中的下一个单词作为预测目标：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [33]:
# ================================================================
# 模块功能说明：
# 使用 tiktoken GPT-2 编码器将完整文本文件编码为 token id 列表
# ================================================================

# ------------------------------------------------
# 1. 读取文本文件
# ------------------------------------------------
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
# - open("the-verdict.txt", "r", encoding="utf-8"):
#     1) 打开当前目录下的 "the-verdict.txt" 文件
#     2) "r" 表示读取模式
#     3) encoding="utf-8" 保证文本按 UTF-8 编码读取
# - f.read():
#     1) 读取文件中所有内容
#     2) 返回字符串类型，保存到变量 raw_text

# ------------------------------------------------
# 2. 使用 tiktoken GPT-2 编码器编码文本
# ------------------------------------------------
enc_text = tokenizer.encode(raw_text)
# - tokenizer: tiktoken.get_encoding("gpt2") 返回的编码器对象
# - encode(raw_text):
#     1) 将文本 raw_text 转换为整数 token id 列表
#     2) 对文本进行 BPE 分词
#     3) 每个 token 对应一个整数 id
# - enc_text:
#     - 一个整数列表，长度等于文本被拆分后的 token 数量
#     - 教学用途：
#         1) 展示大文本在 GPT-2 分词器下被拆分成多少 token
#         2) 为模型输入长度和 token 计数做参考

# ------------------------------------------------
# 3. 输出编码后的 token 数量
# ------------------------------------------------
print(len(enc_text))
# - len(enc_text):
#     1) 计算编码后的整数列表长度
#     2) 即文本被 GPT-2 分词器拆分后的 token 数量
# - 输出示例：
#     15234
# - 教学用途：
#     1) 了解大文本经过 GPT-2 分词后的 token 数量
#     2) 观察 token 数量与原字符数、单词数的关系


5145


- 对于每个文本块，我们需要输入（inputs）和目标（targets）  
- 由于我们希望模型预测下一个单词，目标就是将输入向右移动一个位置


In [34]:
# ================================================================
# 模块功能说明：
# 从编码后的 token 序列中提取子序列（slice）
# ================================================================

# ------------------------------------------------
# 提取 token 子序列
# ------------------------------------------------
enc_sample = enc_text[50:]
# - enc_text: 之前用 tokenizer.encode(raw_text) 编码得到的整数 token 列表
# - [50:]: 切片操作
#     1) 从列表的第 50 个元素（索引 50）开始
#     2) 直到列表末尾
# - enc_sample:
#     - 一个新的整数列表
#     - 包含原始 enc_text 中从第 50 个 token 开始到最后的所有 token
# - 教学用途：
#     1) 演示如何对 token 列表进行切片操作
#     2) 可以用于生成模型输入的子序列或 mini-batch
#     3) 便于观察部分文本编码效果而不处理整个大文本


In [35]:
# ================================================================
# 模块功能说明：
# 演示如何从 token 序列中构建简单的上下文窗口 (context window)
# ================================================================

# ------------------------------------------------
# 1. 定义上下文窗口大小
# ------------------------------------------------
context_size = 4
# - context_size: 整数，表示我们希望取多少个连续 token 作为输入上下文
# - 教学用途：
#     1) 模拟语言模型的输入长度
#     2) 每次用 context_size 个 token 来预测下一个 token

# ------------------------------------------------
# 2. 构建输入序列 x 和目标序列 y
# ------------------------------------------------
x = enc_sample[:context_size]
# - x: 输入序列 (context tokens)
# - enc_sample[:context_size]:
#     1) 切片操作，取 enc_sample 列表的前 context_size 个 token
#     2) 索引范围：[0, context_size-1]
# - 教学用途：
#     - 模型用 x 来预测下一个 token

y = enc_sample[1:context_size+1]
# - y: 目标序列 (target tokens)
# - enc_sample[1:context_size+1]:
#     1) 从索引 1 开始取 context_size 个 token
#     2) 与 x 对应，表示 x 的每个 token 之后的下一个 token
# - 教学用途：
#     - 用于训练语言模型时作为监督目标
#     - 保证 y 与 x 对齐，长度相同，但每个元素是 x 对应的下一个 token

# ------------------------------------------------
# 3. 输出结果
# ------------------------------------------------
print(f"x: {x}")
print(f"y:      {y}")
# - 输出示例：
#   x: [50256, 15496, 11, 703]
#   y: [15496, 11, 703, 345]
# - 教学用途：
#     1) 直观展示上下文与预测目标的对应关系
#     2) x[i] 的下一个 token 对应 y[i]
#     3) 为后续语言模型训练提供基础理解


x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 一个接一个地，预测过程如下所示：


In [36]:
# ================================================================
# 模块功能说明：
# 演示如何使用动态长度的上下文序列来预测下一个 token
# ================================================================

# ------------------------------------------------
# 1. 遍历 1 到 context_size 的长度
# ------------------------------------------------
for i in range(1, context_size+1):
    # range(1, context_size+1):
    # - 从 1 开始，到 context_size 结束（包含 context_size）
    # - 用于动态构建不同长度的上下文
    # - 教学用途：
    #     1) 展示从 1 个 token 到 context_size 个 token 的预测示例
    #     2) 帮助理解语言模型的上下文感受野

    # ------------------------------------------------
    # 2. 提取当前上下文序列
    # ------------------------------------------------
    context = enc_sample[:i]
    # - context: 当前上下文 token 列表
    # - enc_sample[:i]:
    #     1) 切片操作，从第 0 个 token 到第 i-1 个 token
    #     2) 动态生成长度为 i 的上下文
    # - 教学用途：
    #     - 用 context 作为模型输入，预测下一个 token

    # ------------------------------------------------
    # 3. 提取当前预测目标
    # ------------------------------------------------
    desired = enc_sample[i]
    # - desired: 当前上下文对应的目标 token
    # - enc_sample[i]:
    #     1) 第 i 个 token（索引 i）
    #     2) 即 context 序列之后的下一个 token
    # - 教学用途：
    #     - 演示 x → y 对应关系
    #     - x 可以长度不同，y 始终是 context 的下一个 token

    # ------------------------------------------------
    # 4. 输出当前上下文与目标 token
    # ------------------------------------------------
    print(context, "---->", desired)
    # - 输出示例（假设 enc_sample 前几个 token 为 [50256, 15496, 11, 703, 345]）：
    #   [50256] ----> 15496
    #   [50256, 15496] ----> 11
    #   [50256, 15496, 11] ----> 703
    #   [50256, 15496, 11, 703] ----> 345
    # - 教学用途：
    #     1) 直观展示逐步增加上下文长度预测下一个 token 的过程
# ================================================================


[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [37]:
# ================================================================
# 模块功能说明：
# 演示如何将上下文 token 和目标 token 解码为可读文本进行观察
# ================================================================

# ------------------------------------------------
# 1. 遍历 1 到 context_size 的长度
# ------------------------------------------------
for i in range(1, context_size+1):
    # range(1, context_size+1):
    # - 从 1 开始，到 context_size 结束（包含 context_size）
    # - 用于动态生成不同长度的上下文
    # - 教学用途：
    #     1) 观察短上下文到完整上下文对下一个 token 的预测
    #     2) 帮助理解语言模型的上下文感受野

    # ------------------------------------------------
    # 2. 提取当前上下文 token
    # ------------------------------------------------
    context = enc_sample[:i]
    # - context: 当前上下文 token 列表
    # - enc_sample[:i]:
    #     1) 切片操作，从索引 0 到 i-1
    #     2) 动态生成长度为 i 的上下文
    # - 教学用途：
    #     - 用 context 作为模型输入观察语言模型预测目标 token

    # ------------------------------------------------
    # 3. 提取当前目标 token
    # ------------------------------------------------
    desired = enc_sample[i]
    # - desired: 当前上下文对应的目标 token
    # - enc_sample[i]: 索引 i 的 token
    # - 教学用途：
    #     - 对比 context 的下一个 token，用于理解语言模型训练

    # ------------------------------------------------
    # 4. 解码上下文和目标 token 为可读文本
    # ------------------------------------------------
    # 解码上下文 token 列表为文本
    decoded_context = tokenizer.decode(context)
    # - tokenizer.decode(context):
    #     1) 将 token id 列表 context 转回可读文本字符串
    #     2) 子词拼接为原始单词，保留空格和标点

    # 解码单个目标 token 列表为文本
    decoded_desired = tokenizer.decode([desired])
    # - tokenizer.decode([desired]):
    #     1) 注意必须传入列表形式 [desired]
    #     2) 将单个 token id 转为文本字符串
    #     3) 保留 token 对应的字符或单词

    # ------------------------------------------------
    # 5. 输出上下文和目标 token 的可读文本
    # ------------------------------------------------
    print(decoded_context, "---->", decoded_desired)
    # - 输出示例（假设 context_size = 4，enc_sample 前几个 token 对应文本）：
    #   Hello ----> ,
    #   Hello, ----> do
    #   Hello, do ----> you
    #   Hello, do you ----> like
    # - 教学用途：
    #     1) 直观展示“上下文文本 → 下一个 token 文本”的对应关系
    #     2) 帮助理解语言模型训练时 x → y 的逻辑


 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 下一章节中在讲解注意力机制后，我们将处理下一个单词预测  
- 目前，我们实现一个简单的数据加载器，它遍历输入数据集并返回向右移动一位的输入和目标


- 安装并导入 PyTorch（安装提示请参见附录 A）


In [38]:
# ================================================================
# 模块功能说明：
# 演示如何导入 PyTorch 并查看其版本号
# ================================================================

# ------------------------------------------------
# 1. 导入 PyTorch
# ------------------------------------------------
import torch
# - torch: PyTorch 的核心库
# - 提供：
#     1) 张量（tensor）操作和计算
#     2) GPU 加速
#     3) 神经网络构建（torch.nn）
#     4) 自动求导（autograd）
# - 教学用途：
#     1) 确认 PyTorch 已正确安装
#     2) 为后续深度学习任务准备环境

# ------------------------------------------------
# 2. 打印 PyTorch 版本
# ------------------------------------------------
print("PyTorch version:", torch.__version__)
# - torch.__version__:
#     1) 获取当前安装的 PyTorch 版本号（字符串类型）
#     2) 例如："2.1.0+cpu" 或 "2.1.0+cu118"
# - print(...):
#     - 输出示例：
#       PyTorch version: 2.1.0+cu118
# - 教学用途：
#     1) 确认版本，便于复现实验
#     2) 避免因版本差异导致 API 或功能不同


PyTorch version: 2.9.0+cpu


- 我们使用滑动窗口方法，每次位置向右移动 +1：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">


- 创建数据集和数据加载器，从输入文本数据集中提取文本块


In [40]:
# ================================================================
# 模块功能说明：
# 自定义 PyTorch Dataset，用于 GPT 风格语言模型训练
# ================================================================

from torch.utils.data import Dataset, DataLoader
# - Dataset: PyTorch 数据集基类，用于自定义数据加载逻辑
# - DataLoader: 数据加载器，可对 Dataset 进行批量加载、shuffle 和多线程加速

# ================================================================
# 自定义 GPTDatasetV1 类
# ================================================================
class GPTDatasetV1(Dataset):
    # ------------------------------------------------
    # 1. 初始化函数
    # ------------------------------------------------
    def __init__(self, txt, tokenizer, max_length, stride):
        """
        GPTDatasetV1 初始化函数

        参数：
        - txt: str，完整文本字符串，例如书籍全文
        - tokenizer: 分词器对象，需要提供 encode(text, allowed_special) 方法
        - max_length: int，每个训练样本的最大 token 数量（上下文长度）
        - stride: int，滑动窗口步长，用于生成重叠训练样本
        """
        # 初始化存储输入和目标 token 的列表
        self.input_ids = []   # 存储每个训练样本的输入序列 x 的 token id
        self.target_ids = []  # 存储每个训练样本的目标序列 y 的 token id

        # ------------------------------------------------
        # 2. 对整个文本进行编码
        # ------------------------------------------------
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        # - tokenizer.encode(txt, allowed_special={"<|endoftext|>"}):
        #     1) 将完整文本 txt 编码为整数 token id 列表
        #     2) allowed_special 参数：
        #        - {"<|endoftext|>"} 表示保留特殊 token，不拆分或忽略
        #     3) 返回值 token_ids 为列表，例如：
        #        [50256, 15496, 11, 703, 345, ...]
        #     4) 教学用途：
        #        - 展示 BPE 分词器对完整文本的 token 化过程
        #        - 为语言模型训练提供整数序列输入

        # ------------------------------------------------
        # 3. 检查文本长度是否足够
        # ------------------------------------------------
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"
        # - 保证文本 token 数量大于 max_length
        # - 如果 token 数量不足，无法生成至少一个训练样本
        # - 教学用途：
        #     1) 演示数据预处理时常见的边界检查
        #     2) 提示用户文本长度必须满足训练需求

        # ------------------------------------------------
        # 4. 使用滑动窗口生成训练样本
        # ------------------------------------------------
        for i in range(0, len(token_ids) - max_length, stride):
            # - i: 滑动窗口起始索引
            # - range(0, len(token_ids) - max_length, stride):
            #     1) 从 0 到 len(token_ids)-max_length（保证切片长度为 max_length）
            #     2) 步长为 stride，控制样本之间的重叠长度
            #     3) 教学用途：
            #        - 演示滑动窗口如何生成重叠训练样本
            #        - 增加训练样本数量，提高模型学习上下文能力

            # ------------------------------------------------
            # 4.1 构建输入序列 x
            # ------------------------------------------------
            input_chunk = token_ids[i:i + max_length]
            # - 输入序列 token id 列表
            # - 切片范围：[i, i + max_length-1]
            # - 长度固定为 max_length
            # - 教学用途：
            #     - 模型输入 x
            #     - 用于语言模型训练

            # ------------------------------------------------
            # 4.2 构建目标序列 y
            # ------------------------------------------------
            target_chunk = token_ids[i + 1: i + max_length + 1]
            # - 目标序列 token id 列表
            # - 每个 token 对应输入序列的下一个 token
            # - 切片范围：[i+1, i+max_length]
            # - 长度同样为 max_length
            # - 教学用途：
            #     - 模型输出 y（监督信号）
            #     - 体现语言模型的“下一个 token 预测”机制

            # ------------------------------------------------
            # 4.3 将列表转换为 PyTorch 张量并存储
            # ------------------------------------------------
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            # - torch.tensor(list):
            #     1) 将 Python 列表转换为 PyTorch 张量
            #     2) 默认 dtype 为 torch.long（整数）
            #     3) 可以直接用于模型训练
            # - 教学用途：
            #     - 演示数据预处理流程：文本 → token → tensor
            #     - 为 DataLoader 提供可迭代张量数据

    # ------------------------------------------------
    # 5. 返回数据集长度
    # ------------------------------------------------
    def __len__(self):
        return len(self.input_ids)
        # - 返回训练样本数量
        # - DataLoader 会根据此值生成迭代器
        # - 教学用途：
        #     - 展示 Dataset 接口要求
        #     - 核心方法 __len__ 用于迭代样本数量控制

    # ------------------------------------------------
    # 6. 根据索引获取样本
    # ------------------------------------------------
    def __getitem__(self, idx):
        """
        获取指定索引的训练样本
        
        参数：
        - idx: int，样本索引，0 <= idx < len(self.input_ids)

        返回：
        - (input_tensor, target_tensor)
        """
        return self.input_ids[idx], self.target_ids[idx]
        # - 返回值：
        #     1) input_tensor: 输入 token 张量 x
        #     2) target_tensor: 目标 token 张量 y
        # - 教学用途：
        #     - 展示 Dataset 获取样本的标准方法
        #     - DataLoader 调用 __getitem__ 来批量加载训练数据


In [41]:
# ================================================================
# 模块功能说明：
# 创建用于 GPT 语言模型训练的 PyTorch DataLoader
# ================================================================

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """
    创建 DataLoader 用于批量训练 GPT 风格语言模型

    参数：
    - txt: str，完整文本字符串（训练语料）
    - batch_size: int，每个批次包含的样本数量（默认 4）
    - max_length: int，每个训练样本的最大 token 长度（上下文长度，默认 256）
    - stride: int，滑动窗口步长，用于生成重叠样本（默认 128）
    - shuffle: bool，是否对样本顺序进行随机打乱（默认 True）
    - drop_last: bool，是否丢弃最后不足 batch_size 的样本（默认 True）
    - num_workers: int，DataLoader 使用的子进程数量（默认 0，表示使用主进程）
    
    返回：
    - dataloader: PyTorch DataLoader 对象，可迭代生成训练批次
    """

    # ------------------------------------------------
    # 1. 初始化 tokenizer
    # ------------------------------------------------
    tokenizer = tiktoken.get_encoding("gpt2")
    # - tiktoken.get_encoding("gpt2"):
    #     1) 获取 GPT-2 分词器
    #     2) 提供 encode(text) 和 decode(token_ids) 方法
    # - 教学用途：
    #     - 演示如何加载标准 GPT-2 分词器
    #     - 为 dataset 构建 token id 序列

    # ------------------------------------------------
    # 2. 创建 GPTDatasetV1 数据集
    # ------------------------------------------------
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    # - GPTDatasetV1(txt, tokenizer, max_length, stride):
    #     1) 自定义 Dataset，将文本切片为重叠 token 序列
    #     2) dataset.input_ids / dataset.target_ids 包含训练样本
    # - 教学用途：
    #     - 展示文本如何预处理为模型可训练样本
    #     - 输入文本 → token id → dataset 张量序列

    # ------------------------------------------------
    # 3. 创建 PyTorch DataLoader
    # ------------------------------------------------
    dataloader = DataLoader(
        dataset,             # 数据集对象
        batch_size=batch_size, # 每个批次样本数量
        shuffle=shuffle,       # 是否打乱样本顺序
        drop_last=drop_last,   # 是否丢弃最后不足 batch_size 的批次
        num_workers=num_workers # DataLoader 使用的子进程数量
    )
    # - DataLoader:
    #     1) 自动对 Dataset 进行批量采样
    #     2) 支持 shuffle（随机打乱）和多线程加速
    # - 教学用途：
    #     - 展示如何用 Dataset 创建可迭代的训练数据
    #     - 模拟训练循环中常用的数据加载方式

    # ------------------------------------------------
    # 4. 返回 DataLoader
    # ------------------------------------------------
    return dataloader
    # - dataloader:
    #     1) 可迭代对象
    #     2) 每次迭代返回一个 batch: (input_tensor_batch, target_tensor_batch)
    #     3) input_tensor_batch 和 target_tensor_batch 形状为：
    #        [batch_size, max_length]，类型为 torch.LongTensor


- 让我们使用批量大小为 1 来测试数据加载器，LLM 的上下文长度为 4：


In [42]:
# ================================================================
# 模块功能说明：
# 读取文本文件内容到 Python 字符串
# ================================================================

# ------------------------------------------------
# 1. 打开文件
# ------------------------------------------------
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    # - open("the-verdict.txt", "r", encoding="utf-8"):
    #     1) "the-verdict.txt" 是文件路径
    #     2) "r" 表示以只读模式打开文件
    #     3) encoding="utf-8" 表示按 UTF-8 编码读取文本
    #        - 确保非 ASCII 字符（如中文、特殊符号）正确读取
    # - with 语句：
    #     - 上下文管理器，确保文件使用完毕后自动关闭
    #     - 避免忘记调用 f.close() 导致资源泄漏
    # - f:
    #     - 文件对象（file object）
    #     - 提供 read()、readline() 等方法读取文件内容

    # ------------------------------------------------
    # 2. 读取文件全部内容
    # ------------------------------------------------
    raw_text = f.read()
    # - f.read():
    #     1) 读取文件中所有内容，并返回字符串类型
    #     2) 教学用途：
    #         - 将文本内容加载到内存中进行处理
    #         - 方便后续分词、编码、Dataset 构建等操作
    # - raw_text:
    #     - str 类型，保存整个文件的文本
    #     - 可以直接传入 tokenizer.encode 或自定义 Dataset 使用


In [43]:
# ================================================================
# 模块功能说明：
# 使用 create_dataloader_v1 创建 DataLoader 并获取第一个训练批次
# ================================================================

# ------------------------------------------------
# 1. 创建 DataLoader
# ------------------------------------------------
dataloader = create_dataloader_v1(
    raw_text,        # 原始文本字符串
    batch_size=1,    # 每个批次只包含 1 个训练样本
    max_length=4,    # 每个训练样本的 token 序列长度为 4
    stride=1,        # 滑动窗口步长为 1，即每次生成的样本只错开 1 个 token
    shuffle=False    # 不打乱顺序，保证样本顺序与文本顺序一致
)
# - 返回 dataloader: PyTorch DataLoader 对象，可迭代生成训练批次
# - 教学用途：
#     1) 演示如何从文本创建可迭代的数据加载器
#     2) small batch_size 和小 max_length 方便调试和展示

# ------------------------------------------------
# 2. 将 DataLoader 转换为迭代器
# ------------------------------------------------
data_iter = iter(dataloader)
# - iter(dataloader):
#     1) 将 DataLoader 转换为迭代器
#     2) 可使用 next() 获取下一个 batch
# - 教学用途：
#     - 演示 DataLoader 内部是可迭代对象
#     - 方便直接访问 batch 进行调试或教学展示

# ------------------------------------------------
# 3. 获取第一个训练批次
# ------------------------------------------------
first_batch = next(data_iter)
# - next(data_iter):
#     1) 获取迭代器的下一个元素，即第一个 batch
#     2) 返回值类型为元组：
#        (input_tensor_batch, target_tensor_batch)
#     3) 每个 tensor 形状为 [batch_size, max_length]，数据类型 torch.LongTensor
# - 教学用途：
#     - 直观展示训练批次的输入和目标
#     - 帮助学生理解 batch 结构

# ------------------------------------------------
# 4. 打印第一个训练批次
# ------------------------------------------------
print(first_batch)
# - 输出示例：
#     (
#       tensor([[50256, 15496, 11, 703]]),  # input_tensor_batch
#       tensor([[15496, 11, 703, 345]])     # target_tensor_batch
#     )
# - 教学用途：
#     1) 查看输入序列与目标序列对应关系
#     2) 便于理解 GPT 语言模型训练中 x → y 的映射


[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [44]:
# ================================================================
# 模块功能说明：
# 获取 DataLoader 中的第二个训练批次并打印内容
# ================================================================

# ------------------------------------------------
# 1. 从迭代器中获取第二个批次
# ------------------------------------------------
second_batch = next(data_iter)
# - next(data_iter):
#     1) 从 dataloader 迭代器中继续取下一个批次数据
#     2) 因为上一个批次（first_batch）已经取出，
#        所以此处取出的即为第二个批次
# - 返回结果：
#     一个元组 (input_tensor_batch, target_tensor_batch)
# - 详细说明：
#     - input_tensor_batch：模型输入（即上下文 token 序列）
#     - target_tensor_batch：模型目标输出（即预测目标 token 序列）
# - 二者形状均为 [batch_size, max_length]
# - 数据类型：torch.LongTensor（整型张量）
# - 教学要点：
#     - 展示 DataLoader 是可连续迭代的
#     - 理解“滑动窗口”如何在文本上依次滑动形成多个样本
#     - 当 stride=1 时，每个样本只向后滑动一个 token

# ------------------------------------------------
# 2. 打印第二个训练批次内容
# ------------------------------------------------
print(second_batch)
# - 输出示例：
#     (
#       tensor([[15496, 11, 703, 345]]),
#       tensor([[11, 703, 345, 257]])
#     )
# - 说明：
#     - 第一个张量是输入序列 input_tensor_batch
#     - 第二个张量是目标序列 target_tensor_batch
# - 教学用途：
#     1) 观察第二个 batch 的 token 序列变化
#     2) 理解输入与目标的“错位”关系：
#        target 是 input 向右平移一个位置后的结果


[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 示例：使用与上下文长度相等的步长（此处为 4），如下所示：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们还可以创建批量输出  
- 注意，这里我们增加了步长，以避免批次之间的重叠，因为过多重叠可能导致过拟合增加


In [45]:
# ================================================================
# 模块功能说明：
# 使用自定义函数 create_dataloader_v1 创建 DataLoader，
# 从文本中按滑动窗口分块生成训练样本，
# 并打印第一个 batch 的输入张量与目标张量。
# ================================================================


# ------------------------------------------------
# 1. 创建 DataLoader
# ------------------------------------------------
dataloader = create_dataloader_v1(
    raw_text,      # txt: 输入的完整文本内容（之前从 "the-verdict.txt" 读取）
    batch_size=8,  # batch_size: 每个批次包含 8 个样本
                   # -> 每次 DataLoader 返回的 input_ids 和 target_ids
                   #    张量形状为 [8, 4]

    max_length=4,  # max_length: 每个样本的上下文长度（即序列长度为 4 个 token）
                   # -> 语言模型一次看到 4 个 token 来预测第 5 个 token

    stride=4,      # stride: 滑动窗口步长（一次滑动 4 个 token）
                   # -> 表示每个样本之间不重叠
                   # -> 第一个样本是 tokens[0:4]，第二个样本是 tokens[4:8]，以此类推

    shuffle=False  # shuffle=False：不打乱样本顺序，保持按文本顺序生成数据
                   # -> 用于教学演示时方便观察 token 顺序变化
)
# DataLoader 内部会自动分批返回 (input_tensor_batch, target_tensor_batch)
# 每次迭代会得到：
#   input_tensor_batch  -> 模型输入（即上下文 token）
#   target_tensor_batch -> 预测目标（即下一个 token）


# ------------------------------------------------
# 2. 创建迭代器对象
# ------------------------------------------------
data_iter = iter(dataloader)
# - iter(dataloader):
#     将 DataLoader 转换为一个 Python 可迭代对象
# - 用于从 DataLoader 中逐批取出样本（即 batch）
# - 每次执行 next(data_iter) 时，会返回下一个批次的数据


# ------------------------------------------------
# 3. 从迭代器中取出第一个批次的数据
# ------------------------------------------------
inputs, targets = next(data_iter)
# - next(data_iter):
#     从 dataloader 中取出第一个 batch
# - 返回值是一个元组 (inputs, targets)
# - 变量含义：
#     inputs:  shape = [batch_size, max_length]
#               每行是一个样本的输入 token 序列
#     targets: shape = [batch_size, max_length]
#               每行是输入序列右移一位后的目标 token 序列
# - 举例说明：
#     若 inputs 第1行为 [100, 200, 300, 400]
#     则 targets 第1行为 [200, 300, 400, 500]
#     模型任务：根据输入预测目标（即下一个 token）


# ------------------------------------------------
# 4. 打印第一个批次的输入张量
# ------------------------------------------------
print("Inputs:\n", inputs)
# - 打印当前批次的输入 token ID 张量
# - 张量形状：[8, 4]
# - 用于查看 token 序列的整数编码形式


# ------------------------------------------------
# 5. 打印第一个批次的目标张量
# ------------------------------------------------
print("\nTargets:\n", targets)
# - 打印目标 token ID 张量
# - 与 inputs 相同形状，但整体右移一个 token
# - 教学要点：
#     1) 帮助理解“语言模型的训练目标”
#     2) 观察输入与输出的“预测关系”


Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建 token 嵌入


- 数据几乎已经准备好用于 LLM  
- 最后，让我们使用嵌入层将 token 转换为连续的向量表示  
- 通常，这些嵌入层是 LLM 本身的一部分，并会在模型训练过程中更新（训练）


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 假设我们有以下四个输入示例，输入 ID 分别为 2、3、5 和 1（经过分词后）：


In [46]:
# ================================================================
# 模块功能说明：
# 创建一个包含整数序列的 PyTorch 张量（tensor），
# 通常用于模拟 token 的整数编码序列（input_ids）。
# ================================================================

# ------------------------------------------------
# 1. 创建张量（tensor）
# ------------------------------------------------
input_ids = torch.tensor([2, 3, 5, 1])
# - 语法：torch.tensor(data)
# - 参数 data: 
#     一个 Python 列表、元组或嵌套结构（如二维列表）
#     此处为 [2, 3, 5, 1]，即一个长度为 4 的一维整数列表
# - 功能说明：
#     将该列表转换为 PyTorch 的张量对象（Tensor）
# - 默认数据类型：
#     若列表中全为整数，则 dtype 默认为 torch.int64
#     （也称为 torch.long，用于索引或 token ID）
# - 教学背景：
#     在语言模型（如 GPT）中，句子经过分词器 (tokenizer)
#     后会变为整数序列，例如：
#         “Hello world.” → [15496, 995, 13]
#     这些整数即“token IDs”，通常存储在张量中以便送入模型。

# ------------------------------------------------
# 2. 张量的形状与内容
# ------------------------------------------------
# - input_ids 的形状（Shape）：torch.Size([4])
#   表示为 1 维张量（向量），共 4 个元素
# - 张量内容（Values）：[2, 3, 5, 1]
# - 数据类型（dtype）：torch.int64（长整型）
# - 存放设备（device）：
#     默认在 CPU 上创建（除非手动指定 device='cuda'）

# ------------------------------------------------
# 3. 实际应用场景
# ------------------------------------------------
# - 在 NLP 模型训练中，该张量通常代表一个句子的 token 序列：
#     例：
#         input_ids = [2, 3, 5, 1]
#         可能表示 "I like tea <EOS>"
# - 模型会将这些整数查表（embedding lookup）
#   转换成对应的向量嵌入（embeddings），
#   然后送入神经网络进行训练或推理。


- 为简单起见，假设我们有一个仅包含 6 个单词的小词汇表，并且希望创建维度为 3 的嵌入向量：


In [47]:
# ================================================================
# 模块功能说明：
# 创建一个用于词向量映射（embedding）的层（torch.nn.Embedding），
# 用于将离散的 token ID（整数索引）映射为连续的向量表示。
# ================================================================


# ------------------------------------------------
# 1. 定义词表大小（vocab_size）
# ------------------------------------------------
vocab_size = 6
# - 含义：
#     表示词汇表（vocabulary）的总 token 数量。
# - 即模型可以识别的不同“单词”或“符号”的总数。
# - 在此例中，假设词汇表中有 6 个唯一的 token，
#   例如：
#       0 → "Hello"
#       1 → "world"
#       2 → "I"
#       3 → "like"
#       4 → "tea"
#       5 → "<|endoftext|>"
# - 说明：
#     每个 token 都会被分配一个唯一的整数 ID（0 到 5）。


# ------------------------------------------------
# 2. 定义词向量维度（output_dim）
# ------------------------------------------------
output_dim = 3
# - 含义：
#     每个 token 会被映射为一个 3 维向量（embedding vector）。
# - 举例说明：
#     若某个 token ID = 2，
#     则 embedding 层会返回一个形状为 [3] 的张量，
#     例如 [0.12, -0.05, 0.33]。
# - 通常在实际模型中，embedding 维度可能设为 128、256、768 等较大值，
#   但在教学或演示时可用较小值（如 3）方便观察。


# ------------------------------------------------
# 3. 固定随机数种子（manual_seed）
# ------------------------------------------------
torch.manual_seed(123)
# - 含义：
#     固定随机数生成器的初始状态，使结果可重复。
# - PyTorch 中很多层（如 Embedding、Linear）初始化时权重是随机的。
# - 通过设置随机种子：
#     每次运行本代码，embedding 层初始化的参数都会相同，
#     方便教学与调试。


# ------------------------------------------------
# 4. 创建词嵌入层（Embedding Layer）
# ------------------------------------------------
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# - 模块说明：
#     torch.nn.Embedding(num_embeddings, embedding_dim)
#     是 PyTorch 中专门用于“整数索引 → 连续向量”映射的层。
# - 参数解释：
#     num_embeddings = vocab_size
#         表示词汇表中可查找的 token 数量（此处为 6）
#     embedding_dim = output_dim
#         表示每个 token 映射后的向量维度（此处为 3）
# - 内部机制：
#     创建一个形状为 [vocab_size, output_dim] 的可学习参数矩阵：
#         权重矩阵 weight.shape = [6, 3]
#     每一行对应一个 token 的向量表示。
# - 示例：
#     embedding_layer.weight 可能如下（随机初始化）：
#       tensor([[ 0.3374, -0.3352,  0.1015],
#               [-0.2652,  0.2919,  0.0536],
#               [ 0.2631, -0.1948,  0.4081],
#               [ 0.2316, -0.3052, -0.2831],
#               [ 0.1130, -0.1020,  0.4342],
#               [ 0.2748, -0.0529, -0.2417]])
# - 使用方式：
#     当输入 token ID 序列 [2, 3, 5] 传入时，
#     embedding_layer 会自动返回对应行的向量：
#       [[0.2631, -0.1948,  0.4081],
#        [0.2316, -0.3052, -0.2831],
#        [0.2748, -0.0529, -0.2417]]
# - 教学要点：
#     Embedding 层的作用类似一个“查表”操作：
#     它不执行矩阵乘法，仅根据索引提取对应行。


- 这将产生一个 6x3 的权重矩阵：


In [48]:
# -------------------------------------------------------------
# 打印嵌入层（Embedding Layer）的权重矩阵
# -------------------------------------------------------------
# embedding_layer 是一个 torch.nn.Embedding 对象，用于将离散的词 ID 映射为连续的向量表示。
# 每一个“词”或“token”在这个嵌入层中对应一行向量。
# 
# 举例说明：
#   若 vocab_size = 6，表示词汇表中共有 6 个不同的 token（编号 0~5）
#   若 output_dim = 3，表示每个 token 被映射为 3 维的向量
# 因此 embedding_layer.weight 的形状是 [6, 3]
#
# 这些权重最初是随机初始化的（因为我们调用了 torch.manual_seed(123)，所以结果可复现）
# 在训练过程中，这些权重会被不断更新，以便更好地表示不同 token 的语义关系。
#
# 输出示例：
# tensor([[ 0.1339, -0.2116,  0.3899],
#         [-2.2323,  0.1258, -0.4833],
#         [ 1.2467,  0.8545,  0.3032],
#         [ 0.2348, -0.5379,  0.8874],
#         [ 0.1145, -0.2932, -0.1399],
#         [ 0.7143, -0.2890, -1.0458]])
# 每一行表示一个词（token）的向量表示
print(embedding_layer.weight)


Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉独热编码（one-hot encoding）的人来说，上述嵌入层方法本质上只是实现独热编码后进行矩阵乘法的一种更高效方式，这在补充代码 [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul) 中有描述  
- 由于嵌入层只是独热编码加矩阵乘法方法的更高效实现，因此它可以被视为一个神经网络层，可以通过反向传播进行优化


- 要将 ID 为 3 的 token 转换为 3 维向量，我们执行以下操作：


In [49]:
# -------------------------------------------------------------
# 通过嵌入层（Embedding Layer）查询一个 token 的向量表示
# -------------------------------------------------------------
# torch.tensor([3])：
#   创建一个包含单个整数 3 的张量。
#   这个整数 3 表示词汇表中编号为 3 的 token。
#   （假设 vocab_size = 6，则 token 的编号范围为 0~5）
#
# embedding_layer(torch.tensor([3]))：
#   将输入的 token ID 张量传入嵌入层。
#   PyTorch 会自动在 embedding_layer.weight 查找第 3 行的向量，
#   即返回对应 token ID 的嵌入表示。
#
# 输出结果形状为 [1, output_dim]：
#   - “1” 表示输入中有 1 个 token
#   - “output_dim” 表示每个 token 被映射成的向量维度（此处为 3）
#
# 举例说明：
#   假设 embedding_layer.weight 第 3 行为 [0.2348, -0.5379, 0.8874]
#   则输出结果为：
#   tensor([[ 0.2348, -0.5379,  0.8874]])
#
# 这个输出向量就是 token ID = 3 的语义表示（embedding）
print(embedding_layer(torch.tensor([3])))


tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 注意，上述向量对应 `embedding_layer` 权重矩阵的第 4 行  
- 要嵌入上述四个 `input_ids` 的所有值，我们执行：


In [50]:
# -------------------------------------------------------------
# 通过嵌入层（Embedding Layer）将多个 token ID 映射为向量表示
# -------------------------------------------------------------
# embedding_layer(input_ids)：
#   将输入张量 input_ids 中的每个整数（token ID）映射为对应的向量。
#   embedding_layer 会查找 embedding_layer.weight 的对应行。
#
# input_ids：
#   是一个一维张量，例如：
#   tensor([2, 3, 5, 1])
#   表示有 4 个 token，ID 分别为 2、3、5、1。
#
# embedding_layer：
#   是一个 nn.Embedding(vocab_size=6, output_dim=3)
#   - vocab_size = 6：词汇表中共有 6 个不同的 token（编号 0~5）
#   - output_dim = 3：每个 token 被映射成一个 3 维向量
#
# embedding_layer(input_ids) 的计算逻辑：
#   逐个读取 input_ids 中的 token ID，
#   查找 embedding_layer.weight 对应行的向量：
#
#   输出形状为 [len(input_ids), output_dim]：
#       即 [4, 3]
#
#   例如：
#   embedding_layer.weight =
#   tensor([[-0.0869,  0.1345, -0.0892],
#           [ 0.5422, -0.5231, -0.1597],
#           [-0.2610, -0.2392,  0.4909],
#           [ 0.3217, -0.1079,  0.4561],
#           [ 0.1347,  0.2301, -0.5634],
#           [-0.3382,  0.3178,  0.1586]])
#
#   input_ids = tensor([2, 3, 5, 1])
#   则输出：
#   tensor([[-0.2610, -0.2392,  0.4909],   # ID 2
#           [ 0.3217, -0.1079,  0.4561],   # ID 3
#           [-0.3382,  0.3178,  0.1586],   # ID 5
#           [ 0.5422, -0.5231, -0.1597]])  # ID 1
#
# 总结：
#   - 输入：token ID 序列 → tensor([2, 3, 5, 1])
#   - 输出：每个 token 对应的向量 → 形状 [4, 3]
#   - 这个过程相当于查表（Look-up Table）
print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上就是一个查找操作：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **你可能会对比较嵌入层与普通线性层的补充内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**


## 2.8 编码单词位置


- 嵌入层将 ID 转换为向量表示时，不考虑它们在输入序列中的位置：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置嵌入（Positional embeddings）与 token 嵌入向量结合，形成大语言模型的输入嵌入：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- BytePair 编码器的词汇表大小为 50,257  
- 假设我们希望将输入 token 编码为 256 维向量表示：


In [51]:
# -------------------------------------------------------------
# 定义 GPT 模型的“词嵌入层”（Token Embedding Layer）
# -------------------------------------------------------------
# nn.Embedding 是 PyTorch 中的嵌入层（Embedding Layer）类，
# 作用：将“整数形式的 token ID”映射为“连续的向量表示”。
# 这一步是 Transformer / GPT 模型中将文本转为数值向量的关键步骤。
# -------------------------------------------------------------

# 定义词汇表大小（vocab_size）
vocab_size = 50257
# 含义：
#   - GPT-2 的词汇表大小为 50257（包括普通词、标点符号、特殊标记等）
#   - 每个 token 都有一个唯一的 ID（范围：0 ~ 50256）
#   - 例如：'Hello' → 15496,  'world' → 2159,  '<|endoftext|>' → 50256

# 定义嵌入向量的维度（output_dim）
output_dim = 256
# 含义：
#   - 每个 token 将被映射为一个 256 维的向量
#   - 这个维度控制模型的容量（维度越高，模型能表达的信息越多，但计算量也更大）
#   - GPT-2 的 small 版本通常使用 768 维，这里用 256 维是简化演示版

# 创建嵌入层（Token Embedding Layer）
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# 参数解释：
#   - vocab_size：嵌入矩阵的“行数”，即词汇表大小（50257）
#   - output_dim：嵌入矩阵的“列数”，即每个 token 的向量长度（256）
#
# 内部机制：
#   - embedding_matrix = nn.Parameter(torch.empty(vocab_size, output_dim))
#     也就是一个形状为 [50257, 256] 的矩阵
#   - 每个 token ID 对应 embedding_matrix 的一行
#   - 当我们输入 token ID（如 tensor([15496, 2159])）时，
#     嵌入层会查找 embedding_matrix 中第 15496 行和第 2159 行，
#     输出这两行对应的向量。
#
# 输出形状：
#   输入形状：[sequence_length]
#   输出形状：[sequence_length, output_dim]
#
# 举例说明：
#   输入：tensor([15496, 2159])   # ['Hello', 'world']
#   输出：tensor([[v1, v2, ..., v256],
#                 [w1, w2, ..., w256]])   # 形状 [2, 256]


- 如果从数据加载器中采样数据，我们将每个批次中的 token 嵌入为 256 维向量  
- 如果批量大小为 8，每个批次有 4 个 token，则得到一个 8 x 4 x 256 的张量：


In [52]:
# ================================================================
# 通过自定义函数 create_dataloader_v1 创建训练 DataLoader
# ================================================================

# -------------------------------------------------------------
# 1. 设置上下文长度（max_length）
# -------------------------------------------------------------
max_length = 4
# - max_length 表示每个训练样本的上下文长度（sequence length）
# - 对于语言模型：
#     输入序列长度 = max_length
#     输出目标序列长度 = max_length（右移一个 token）
# - 例如：
#     输入 token 序列：[t1, t2, t3, t4]
#     目标 token 序列：[t2, t3, t4, t5]


# -------------------------------------------------------------
# 2. 创建 DataLoader
# -------------------------------------------------------------
dataloader = create_dataloader_v1(
    raw_text,           # txt: 完整文本数据
    batch_size=8,       # batch_size: 每批次包含 8 个样本
    max_length=max_length,  # 每个样本序列长度为 4
    stride=max_length,      # stride = max_length
                            # -> 样本之间不重叠
    shuffle=False        # 不打乱样本顺序
)
# DataLoader 功能：
#   - 根据文本和滑动窗口生成输入序列和目标序列
#   - 每次迭代返回一个 batch: (inputs, targets)
#   - inputs.shape = [batch_size, max_length]
#   - targets.shape = [batch_size, max_length]


# -------------------------------------------------------------
# 3. 创建迭代器
# -------------------------------------------------------------
data_iter = iter(dataloader)
# - 将 DataLoader 转换为迭代器
# - 可以通过 next() 逐批获取训练数据


# -------------------------------------------------------------
# 4. 获取第一个批次数据
# -------------------------------------------------------------
inputs, targets = next(data_iter)
# - inputs: [batch_size, max_length] 的张量，表示输入 token 序列
# - targets: [batch_size, max_length] 的张量，表示右移一个 token 的目标序列
# - 举例说明：
#     若 batch_size=8，max_length=4：
#     inputs.shape = [8, 4], targets.shape = [8, 4]
#     每一行表示一个样本序列


In [53]:
# -------------------------------------------------------------
# 打印输入的 token ID 张量
# -------------------------------------------------------------
# inputs 是从 DataLoader 中获取的一个批次输入
# 形状为 [batch_size, max_length]，每个元素是一个整数 token ID
# 打印示例：
# tensor([[15496,   995,    13,  50256],
#         [  2159,  1337,   432,  50256],
#         ...])
print("Token IDs:\n", inputs)


# -------------------------------------------------------------
# 打印 inputs 张量的形状
# -------------------------------------------------------------
# inputs.shape:
# - 输出张量的维度信息
# - 形状为 [batch_size, max_length]
# - 举例：
#     batch_size = 8, max_length = 4
#     则输出 torch.Size([8, 4])
print("\nInputs shape:\n", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [56]:
# ================================================================
# 将输入的 token IDs 转换为向量表示（Token Embeddings）
# ================================================================

# -------------------------------------------------------------
# 1. 使用词嵌入层（Embedding Layer）将 token IDs 映射为向量
# -------------------------------------------------------------
token_embeddings = token_embedding_layer(inputs)
# - token_embedding_layer: nn.Embedding(vocab_size, output_dim)
#   - vocab_size = 50257
#   - output_dim = 256
# - inputs: [batch_size, max_length] 的整数张量
# - token_embeddings 的输出：
#   - 每个 token ID 对应 embedding_matrix 的一行向量
#   - 输出张量形状 = [batch_size, max_length, output_dim]
#   - 举例：
#       inputs.shape = [8, 4]
#       output token_embeddings.shape = [8, 4, 256]

# -------------------------------------------------------------
# 2. 打印 token_embeddings 的形状
# -------------------------------------------------------------
# 了解嵌入结果的维度信息
print(token_embeddings.shape)
# 输出示例：
# torch.Size([8, 4, 256])
# - 8: 批次大小（batch_size）
# - 4: 序列长度（max_length）
# - 256: 每个 token 的嵌入向量维度（output_dim）

# -------------------------------------------------------------
# 3. 打印具体的嵌入向量
# -------------------------------------------------------------
# - token_embeddings 中的每个元素都是一个 256 维向量
# - 仅用于教学观察，实际训练时通常不直接打印
print(token_embeddings)


torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

- GPT-2 使用绝对位置嵌入，因此我们只需创建另一个嵌入层：


In [57]:
# ================================================================
# 创建位置嵌入层（Positional Embedding Layer）
# ================================================================

# -------------------------------------------------------------
# 1. 设置上下文长度（context_length）
# -------------------------------------------------------------
context_length = max_length
# - context_length 表示序列的最大长度
# - 位置嵌入的作用：为序列中的每个位置添加唯一的向量表示
# - 举例：
#     如果 max_length = 4，则 context_length = 4
#     对应 4 个位置的嵌入：位置 0、1、2、3

# -------------------------------------------------------------
# 2. 创建位置嵌入层
# -------------------------------------------------------------
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# - 参数解释：
#   - num_embeddings = context_length
#       表示有多少个不同的位置（0 ~ context_length-1）
#   - embedding_dim = output_dim
#       每个位置对应的向量维度（与 token embedding 维度相同，便于相加）
# - 输出：
#   pos_embedding_layer.weight.shape = [context_length, output_dim]
# - 内部机制：
#   - 为每个位置生成一个可学习的向量
#   - 在 Transformer/GPT 模型中，位置向量会与 token 嵌入向量相加
#     从而保留序列位置信息

# -------------------------------------------------------------
# 3. 打印位置嵌入层权重
# -------------------------------------------------------------
# - 每一行表示一个位置（0~context_length-1）的向量
# - 用于教学观察
print(pos_embedding_layer.weight)


Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       requires_grad=True)


In [58]:
# ================================================================
# 获取序列中每个位置的嵌入向量（Positional Embeddings）
# ================================================================

# -------------------------------------------------------------
# 1. 生成位置索引张量
# -------------------------------------------------------------
# torch.arange(max_length)：
# - 创建一个从 0 到 max_length-1 的整数序列
# - 形状：[max_length]
# - 举例：
#     如果 max_length = 4，则 torch.arange(max_length) = tensor([0, 1, 2, 3])
# - 每个整数表示序列中的一个位置 ID
#   位置 0 对应第一个 token，位置 1 对应第二个 token，以此类推

# -------------------------------------------------------------
# 2. 查询位置嵌入
# -------------------------------------------------------------
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
# - pos_embedding_layer: nn.Embedding(context_length, output_dim)
# - 输入：位置 ID 张量 [0, 1, 2, ..., max_length-1]
# - 输出：
#   - 每个位置 ID 对应一个 output_dim 维向量
#   - 输出形状：[max_length, output_dim]
#   - 举例：
#       max_length = 4, output_dim = 256
#       pos_embeddings.shape = [4, 256]
# - 作用：
#   - 给每个 token 添加唯一的位置信息
#   - 在 GPT/Transformer 中，通常将 token_embedding + pos_embedding 作为最终输入

# -------------------------------------------------------------
# 3. 打印 pos_embeddings 的形状
# -------------------------------------------------------------
print(pos_embeddings.shape)
# 输出示例：
# torch.Size([4, 256])
# - 4: 序列长度（max_length）
# - 256: 嵌入维度（output_dim）

# -------------------------------------------------------------
# 4. 打印具体位置嵌入向量
# -------------------------------------------------------------
# - pos_embeddings 中每一行是一个位置向量
# - 用于教学观察
print(pos_embeddings)


torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


| 名称                    | 类型           | 形状                           | 用途                  |
| --------------------- | ------------ | ---------------------------- | ------------------- |
| `pos_embedding_layer` | nn.Embedding | [context_length, output_dim] | 可训练层本身，存储位置向量       |
| `pos_embeddings`      | torch.Tensor | [max_length, output_dim]     | 经过查表后的具体位置向量，用于模型计算 |


一句话理解：

pos_embedding_layer 是“位置向量表”（表格 + 可训练），

pos_embeddings 是“具体取出的行向量”，可以直接与 token embedding 相加。


- 要创建 LLM 使用的输入嵌入，我们只需将 token 嵌入与位置嵌入相加：


In [59]:
# ================================================================
# 将 token 嵌入与位置嵌入相加，形成最终输入向量
# ================================================================

# -------------------------------------------------------------
# 1. 形状对齐
# -------------------------------------------------------------
# token_embeddings.shape = [batch_size, max_length, output_dim]
# pos_embeddings.shape   = [max_length, output_dim]
# 为了能相加，PyTorch 会自动广播（broadcasting）：
#   pos_embeddings 会在 batch_size 维度上复制
#   形状对齐后：[batch_size, max_length, output_dim]

# -------------------------------------------------------------
# 2. 相加
# -------------------------------------------------------------
input_embeddings = token_embeddings + pos_embeddings
# - 每个 token 的最终输入向量 = token embedding + position embedding
# - 这样模型既知道 token 的语义，也知道 token 在序列中的位置
# - output shape = [batch_size, max_length, output_dim]
#   - batch_size: 批次大小
#   - max_length: 序列长度
#   - output_dim: 嵌入向量维度

# -------------------------------------------------------------
# 3. 打印形状
# -------------------------------------------------------------
print(input_embeddings.shape)
# 输出示例：
# torch.Size([8, 4, 256])
# - 8: batch_size
# - 4: max_length
# - 256: embedding dimension

# -------------------------------------------------------------
# 4. 打印具体向量
# -------------------------------------------------------------
# - input_embeddings 每一行是 token embedding + position embedding
# - 用于模型输入
print(input_embeddings)


torch.Size([8, 4, 256])
tensor([[[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
         [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
         [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
         [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],

        [[ 3.2835,  1.1749, -1.4150,  ..., -0.3281,  2.4332,  0.6924],
         [-0.2199, -0.9114, -0.1750,  ...,  1.5337, -0.1998,  0.1462],
         [ 1.5197, -1.4240,  0.4391,  ...,  1.0494, -1.4318,  2.3057],
         [ 0.2893,  0.8346, -0.1884,  ...,  1.9602,  0.8709,  0.8796]],

        [[ 0.9662,  0.0952, -0.4640,  ..., -1.0320,  1.6290,  1.7771],
         [ 2.4468, -0.2154,  1.4984,  ...,  1.8766,  0.5595, -0.1423],
         [-0.3856, -2.5393,  1.1556,  ...,  3.6157,  1.3267,  0.4944],
         [-0.2487, -0.5275,  2.0009,  ...,  0.2930,  0.5977,  1.3300]],

        ...,

        [[ 0.1219,  0.3991, -3.2740,  ..., -1.1921,  2.6637,  2.6728],
         [ 1.2438, -1.6436, -1.11

- 在输入处理工作流程的初始阶段，输入文本被分割成单独的 token  
- 在分割之后，这些 token 会根据预定义词汇表转换为 token ID：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# 总结与收获


请参阅 [./dataloader.ipynb](./dataloader.ipynb) 代码笔记本，这是本章中实现的数据加载器的简明版本，在后续章节训练 GPT 模型时会用到。

请参阅 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 获取练习题答案。

如果你有兴趣了解 GPT-2 的 tokenizer 如何从零实现和训练，请参阅 [Byte Pair Encoding (BPE) Tokenizer From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) 笔记本。
